# MRI based brain tumor IDH classification with MONAI (3D multiparametric MRI)

This tutorial shows how to construct a training workflow of binary classification task.  
And it contains below features:
1. Transforms for Monai dictionary format data.
2. Define a new transform according MONAI transform API.
3. Load Nifti image with metadata, load a list of images and stack them.
5. 3D Voxel DynUNet model, Dice loss, cross entropy loss function for IDH classification task.
6. Deterministic training for reproducibility.

The Brain tumor dataset can be downloaded from 
https://ipp.cbica.upenn.edu/ and  http://medicaldecathlon.com/.  

Target: IDH classification based on whole brain, tumour core, whole tumor, and enhancing tumor from MRI 
Modality: Multimodal multisite MRI data (FLAIR, T1w, T1gd,T2w)  
training: 135 3D MRI \
validation:  \
testing: Not revealed

Source: BRATS 2020/2021 datasets.  
Challenge: RSNA-MICCAI Brain Tumor Radiogenomic Classification

Below figure shows image patches with the tumor sub-regions that are annotated in the different modalities (top left) and the final labels for the whole dataset (right). (Figure taken from the [BraTS IEEE TMI paper](https://ieeexplore.ieee.org/document/6975210/))  
![image](https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/42/7283692/6975210/6975210-fig-3-source-large.gif)

The image patches show from left to right:
1. the whole tumor (yellow) visible in T2-FLAIR (Fig.A).
2. the tumor core (red) visible in T2 (Fig.B).
3. the enhancing tumor structures (light blue) visible in T1Gd, surrounding the cystic/necrotic components of the core (green) (Fig. C).
4. The segmentations are used to generate the final labels of the tumor sub-regions (Fig.D): edema (yellow), non-enhancing solid core (red), necrotic/cystic core (green), enhancing core (blue).

In [1]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import shutil
import tempfile
import sys
import gc
import logging
import copy
import pdb

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import seaborn as sns
import numpy as np
import pandas as pd
import scipy
from scipy import ndimage
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.nn.functional as F
from monai.networks.nets import DynUNet, EfficientNetBN, DenseNet121, SegResNet, SegResNetVAE
from monai.data import CacheDataset, Dataset, DataLoader, ThreadDataLoader, list_data_collate
from torch.utils.data import WeightedRandomSampler

import monai
from monai.transforms import (
    Activations,
    AsDiscrete,
    CastToTyped,
    Compose, 
    CropForegroundd,
    ResizeWithPadOrCrop,
    ResizeWithPadOrCropd,
    Spacingd,
    RandRotate90d,
    Resized,
    EnsureChannelFirstd, 
    Orientationd,
    LoadImaged,
    CopyItemsd,
    NormalizeIntensity,
    HistogramNormalize,
    NormalizeIntensityd,
    RandCropByPosNegLabeld,
    RandCropByLabelClassesd,
    RandAffined,
    RandFlipd,
    Flipd,
    RandGaussianNoised,
    RandGaussianSmoothd,
    RandGibbsNoised,
    RandStdShiftIntensityd,
    RandScaleIntensityd,
    RandZoomd, 
    SpatialCrop, 
    SpatialPadd, 
    MapTransform,
    CastToType,
    ToTensord,
    AddChanneld,
    MapTransform,
    Orientationd,
    ScaleIntensityd,
    ScaleIntensity,
    ScaleIntensityRangePercentilesd,
    KeepLargestConnectedComponentd,
    KeepLargestConnectedComponent,
    ScaleIntensityRange,
    RandShiftIntensityd,
    RandAdjustContrastd,
    AdjustContrastd,
    Rotated,
    ToNumpyd,
    ToDeviced,
    EnsureType,
    EnsureTyped,
    DataStatsd,
)

from monai.config import KeysCollection
from monai.transforms.compose import MapTransform, Randomizable
from collections.abc import Iterable
from typing import Any, Dict, Hashable, Mapping, Optional, Sequence, Tuple, Union
from monai.utils import set_determinism
from monai.utils import (
    ensure_tuple,
    ensure_tuple_rep,
    ensure_tuple_size,
)

from monai.optimizers import LearningRateFinder

from monai.transforms.compose import MapTransform
from monai.transforms.utils import generate_spatial_bounding_box
from skimage.transform import resize
from monai.losses import DiceCELoss, DiceLoss
from monai.utils import set_determinism
from monai.inferers import sliding_window_inference


from monai.metrics import DiceMetric, ROCAUCMetric, HausdorffDistanceMetric
from monai.data import decollate_batch
import glob
import monai
from monai.metrics import compute_meandice
import random
import pickle
from collections import OrderedDict
from typing import Sequence, Optional
import ipywidgets as widgets
from itertools import compress
import SimpleITK as sitk
import torchio as tio

import sklearn
from sklearn.metrics import mean_squared_error, roc_auc_score, accuracy_score, recall_score, \
accuracy_score, precision_score, f1_score, make_scorer, balanced_accuracy_score 

from monai.utils import ensure_tuple_rep
from monai.networks.layers.factories import Conv, Dropout, Norm, Pool
import matplotlib.pyplot as plt
from ranger21 import Ranger21

### monai and ignite based imports
import logging
import sys
from ignite.engine import Engine, Events
from ignite.contrib.handlers import FastaiLRFinder, ProgressBar
from ignite.engine import (
    Events,
    _prepare_batch,
    create_supervised_evaluator,
    create_supervised_trainer,
)
from ignite.handlers import EarlyStopping, ModelCheckpoint


from tqdm import tqdm
from itkwidgets import view
import random
monai.config.print_config()
#from sliding_window_inference_classes import sliding_window_inference_classes

MONAI version: 0.9.0
Numpy version: 1.22.3
Pytorch version: 1.10.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: af0e0e9f757558d144b655c63afcea3a4e0a06f5
MONAI __file__: /home/mmiv-ml/anaconda3/envs/sa_tumorseg22/lib/python3.9/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.9
Nibabel version: 4.0.1
scikit-image version: 0.19.3
Pillow version: 9.0.1
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.11.2
tqdm version: 4.64.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.1
pandas version: 1.4.2
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-rec

In [2]:
MAX_THREADS =2
sitk.ProcessObject.SetGlobalDefaultNumberOfThreads(MAX_THREADS)

In [3]:
seeds = 40961024
set_determinism(seed=seeds)
##np.random.seed(seeds) np random seed does not work here
!nvidia-smi

Mon Jul  4 00:37:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.172.01   Driver Version: 450.172.01   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   38C    P0    41W / 300W |    108MiB / 32505MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [4]:
#patch_size = (128, 128, 128)
spacing = (1.0, 1.0, 1.0)
os.environ["CUDA_VISIBLE_DEVICES"] ="2"
device = torch.device('cuda:0')
deviceName = 'cuda:0'

In [5]:
pd.set_option('display.max_colwidth', None)
data_rpath = '/home/mmiv-ml/data'

In [6]:
BraTS20Subjectsp1q19WithMetaDF  = pd.read_csv('assets/BraTS_TCGA_LGG_GBM_LGG_1p19qDFMetaFew.csv')
BraTS20Subjectsp1q19WithMetaDF

,BraTS2021,t1wPath,t1cwPath,t2wPath,flairPath,segPath,brain_maskPath,brain_mask_ch2Path,brain_mask_ch1Path,BraTS2020,...,Histology,Grade,Data Collection,IDH,1p19q_co_deletion_bin,IDH,1p19q_co_deletion,CV_group,is_merged_2,is_merged_0
0,BraTS2021_00140,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_00140/ROI_BraTS2021_00140.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_00140/BraTS2021_00140_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_00140/BraTS2021_00140_BrainROI.nii.gz,BraTS20_Training_233,...,glioblastoma,G4,HGG,WT,0.0,NaN,non-codel,3.0,both,NaN
1,BraTS2021_01283,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01283/ROI_BraTS2021_01283.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01283/BraTS2021_01283_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01283/BraTS2021_01283_BrainROI.nii.gz,BraTS20_Training_165,...,glioblastoma,G4,HGG,WT,0.0,NaN,non-codel,2.0,both,NaN
2,BraTS2021_01528,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01528/ROI_BraTS2021_01528.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01528/BraTS2021_01528_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01528/BraTS2021_01528_BrainROI.nii.gz,BraTS20_Training_323,...,oligoastrocytoma,G2,LGG,Mutant,0.0,IDH1,non-codel,3.0,both,NaN
3,BraTS2021_01503,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01503/ROI_BraTS2021_01503.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01503/BraTS2021_01503_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01503/BraTS2021_01503_BrainROI.nii.gz,BraTS20_Training_298,...,oligodendroglioma,G3,LGG,Mutant,1.0,IDH1,codel,3.0,both,NaN
4,BraTS2021_01453,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01453/BraTS2021_01453_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01453/BraTS2021_01453_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS20

In [7]:
BraTS20Subjectsp1q19WithMetaDF = BraTS20Subjectsp1q19WithMetaDF.apply(pd.to_numeric, errors = 'ignore')
BraTS20Subjectsp1q19WithMetaDF

,BraTS2021,t1wPath,t1cwPath,t2wPath,flairPath,segPath,brain_maskPath,brain_mask_ch2Path,brain_mask_ch1Path,BraTS2020,...,Histology,Grade,Data Collection,IDH,1p19q_co_deletion_bin,IDH,1p19q_co_deletion,CV_group,is_merged_2,is_merged_0
0,BraTS2021_00140,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_00140/ROI_BraTS2021_00140.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_00140/BraTS2021_00140_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_00140/BraTS2021_00140_BrainROI.nii.gz,BraTS20_Training_233,...,glioblastoma,G4,HGG,WT,0.0,NaN,non-codel,3.0,both,NaN
1,BraTS2021_01283,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01283/ROI_BraTS2021_01283.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01283/BraTS2021_01283_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01283/BraTS2021_01283_BrainROI.nii.gz,BraTS20_Training_165,...,glioblastoma,G4,HGG,WT,0.0,NaN,non-codel,2.0,both,NaN
2,BraTS2021_01528,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01528/ROI_BraTS2021_01528.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01528/BraTS2021_01528_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01528/BraTS2021_01528_BrainROI.nii.gz,BraTS20_Training_323,...,oligoastrocytoma,G2,LGG,Mutant,0.0,IDH1,non-codel,3.0,both,NaN
3,BraTS2021_01503,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01503/ROI_BraTS2021_01503.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01503/BraTS2021_01503_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01503/BraTS2021_01503_BrainROI.nii.gz,BraTS20_Training_298,...,oligodendroglioma,G3,LGG,Mutant,1.0,IDH1,codel,3.0,both,NaN
4,BraTS2021_01453,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01453/BraTS2021_01453_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01453/BraTS2021_01453_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS20

In [8]:
np.unique(BraTS20Subjectsp1q19WithMetaDF['1p19q_co_deletion_bin'].values, return_counts = True)
#BraTS20SubjectsIDHWithMetaDF['1p19q_co_deletion_bin'].values

(array([0., 1.]), array([238, 130]))

In [9]:
BraTS20Subjectsp1q19WithMetaDF.dtypes

BraTS2021                          object
t1wPath                            object
t1cwPath                           object
t2wPath                            object
flairPath                          object
segPath                            object
brain_maskPath                     object
brain_mask_ch2Path                 object
brain_mask_ch1Path                 object
BraTS2020                          object
BraTS2019                          object
BraTS2018                          object
BraTS2017                          object
CohortName                         object
SiteNo_Originating_Institution    float64
TCGA_ID                            object
is_merged_1                        object
Age                               float64
Gender                             object
Histology                          object
Grade                              object
Data Collection                    object
   IDH                             object
1p19q_co_deletion_bin             

## Cearing a list of dictionaries in order to feed into Monai's Dataset
Keys:
- ***image:*** T1, T1c, T2, and flair image
- ***label:*** Segmented mask GT
- ***brain_mask:*** Whole brain area (brain area=1 and Non brain area=0)
- ***IDH_value:*** IDH class corresponding to the subject/images


### Creating/extracting 3 splits for cross validaion (3 cross validaion)

In [10]:
def get_train_val_test_df(BraTS20SubjectsIDHWithMetaDF):
    
    
    BraTS20SubjectsIDHTrainDCT = {}
    BraTS20SubjectsIDHValDCT = {}
    BraTS20SubjectsIDHTestDCT = {}
    
    aDCT = {'fold0':[1, 2, 3], 'fold1':[2, 3, 1], 'fold3': [3, 1, 2]}
    
    for indx, (akey, aval) in enumerate(aDCT.items()):
        
    
        BraTS20SubjectsIDHWithMetaDFTrain = BraTS20SubjectsIDHWithMetaDF.loc[BraTS20SubjectsIDHWithMetaDF['CV_group']==aval[0]]
        BraTS20SubjectsIDHWithMetaDFVal = BraTS20SubjectsIDHWithMetaDF.loc[BraTS20SubjectsIDHWithMetaDF['CV_group']==aval[1]]
        BraTS20SubjectsIDHWithMetaDFTest = BraTS20SubjectsIDHWithMetaDF.loc[BraTS20SubjectsIDHWithMetaDF['CV_group']==aval[2]]

        train_files = [{'image': (image_nameT1ce, image_nameT2), 'label': label_name, 'brain_mask':brain_mask, 'IDH_label':np.array(IDH_label_name).astype(np.float32)} 
                       for image_nameT1ce, image_nameT2, label_name, brain_mask, IDH_label_name 
                       in zip(BraTS20SubjectsIDHWithMetaDFTrain['t1cwPath'], BraTS20SubjectsIDHWithMetaDFTrain['t2wPath'], BraTS20SubjectsIDHWithMetaDFTrain['segPath'], \
                              BraTS20SubjectsIDHWithMetaDFTrain['brain_mask_ch2Path'], BraTS20SubjectsIDHWithMetaDFTrain['1p19q_co_deletion_bin'].values)]
        
        val_files =[{'image': (image_nameT1ce, image_nameT2), 'label': label_name, 'brain_mask':brain_mask, 'IDH_label':np.array(IDH_label_name).astype(np.float32)} 
                    for image_nameT1ce, image_nameT2, label_name, brain_mask, IDH_label_name 
                    in zip(BraTS20SubjectsIDHWithMetaDFTrain['t1cwPath'], BraTS20SubjectsIDHWithMetaDFVal['t2wPath'],BraTS20SubjectsIDHWithMetaDFVal['segPath'],\
                           BraTS20SubjectsIDHWithMetaDFVal['brain_mask_ch2Path'], BraTS20SubjectsIDHWithMetaDFVal['1p19q_co_deletion_bin'].values)]
        
        test_files = [{'image': (image_nameT1ce, image_nameT2), 'label': label_name, 'brain_mask':brain_mask, 'IDH_label':np.array(IDH_label_name).astype(np.float32)} 
                      for image_nameT1ce, image_nameT2, label_name, brain_mask, IDH_label_name 
                      in zip(BraTS20SubjectsIDHWithMetaDFTrain['t1cwPath'], BraTS20SubjectsIDHWithMetaDFTest['t2wPath'], BraTS20SubjectsIDHWithMetaDFTest['segPath'], \
                             BraTS20SubjectsIDHWithMetaDFTest['brain_mask_ch2Path'], BraTS20SubjectsIDHWithMetaDFTest['1p19q_co_deletion_bin'].values)]
        
        
        BraTS20SubjectsIDHTrainDCT[f'fold{indx}'] = copy.deepcopy(train_files)
        BraTS20SubjectsIDHValDCT[f'fold{indx}'] = copy.deepcopy(val_files)
        BraTS20SubjectsIDHTestDCT[f'fold{indx}'] = copy.deepcopy(test_files)
        
        
        
    return BraTS20SubjectsIDHTrainDCT, BraTS20SubjectsIDHValDCT, BraTS20SubjectsIDHTestDCT
        
        
        
BraTS20SubjectsIDHTrainDCT, BraTS20SubjectsIDHValDCT, BraTS20SubjectsIDHTestDCT =  get_train_val_test_df(BraTS20Subjectsp1q19WithMetaDF)      
        
        

# train_files_image = [(image_nameT1, image_nameT1ce, image_nameT2, image_nameFl) 
#                      for image_nameT1,image_nameT1ce, image_nameT2, image_nameFl 
#                      in zip(dfTrainLbl['t1wPath'], dfTrainLbl['t1cwPath'], dfTrainLbl['T2wPath'], dfTrainLbl['FlairPath'])]
# train_files_label = dfTrainLbl['segPath'].tolist()
# train_files_brain_mask = dfTrainLbl['brain_maskPath'].tolist()
# train_files_IDH_label = dfTrainLbl['IDH_value'].values.ravel().tolist()


In [11]:
# n_splits = 3
# #train_index = np.linspace(0, train_features.shape[0]-1, num = train_features.shape[0], dtype = np.uint16, endpoint=True)
# #partition_data = monai.data.utils.partition_dataset_classes(train_index, train_labels.values.ravel().tolist(), shuffle=True, num_partitions=n_splits) 
# #partition_data = monai.data.utils.partition_dataset_classes(train_files, dfTrainLbl['IDH_value'].values.ravel().tolist(), shuffle=True, num_partitions=n_splits)
# partition_data = monai.data.partition_dataset_classes(train_files, BraTS20SubjectsIDHWithMetaDF['IDH_value'].values.ravel().tolist(), shuffle=True, num_partitions=n_splits)
# print(len(partition_data), len(partition_data[0]), len(partition_data[1]), len(partition_data[2]))


# # val_folds = {}
# # train_folds = {}
# # flds = np.linspace(0, n_splits, num=n_splits, dtype = np.int8)
# # for cfold in range(n_splits):
# #     not_cfold = np.delete(flds, cfold)
# #     val_folds[cfold] = partition_data[cfold]
# # #     train_folds[cfold] = 
# # # sub_flds = flds[..., ~0]   
# # # sub_flds

# val_folds = {}
# train_folds = {}
# flds = np.linspace(0, n_splits, num=n_splits, dtype = np.uint8)
# for cfold in range(n_splits):
#     #val_folds[f"fold{cfold}"] = train_features.values[partition_data[cfold],:]
#     #train_folds[f"fold{cfold}"] = np.delete(train_features.values, partition_data[cfold], axis=0)
#     #not_cfold = np.delete(flds, cfold)
    
#     val_folds[f"fold{cfold}"] = copy.deepcopy(partition_data[cfold])
#     val_folds[f"fold{cfold}_IDH_label"] = copy.deepcopy([adct['IDH_label'].item() for adct in partition_data[cfold]])
#     train_folds_masks = [1]*n_splits
#     train_folds_masks[cfold] = 0
#     partition_data_non_cfold = list()
#     for aDctLstitem in compress(partition_data, train_folds_masks):
#         partition_data_non_cfold.extend(aDctLstitem)
        
        
#     train_folds[f"fold{cfold}"] = copy.deepcopy(partition_data_non_cfold)
#     train_folds[f"fold{cfold}_IDH_label"] = copy.deepcopy([adct['IDH_label'].item() for adct in partition_data_non_cfold])

# for i in range(n_splits):
#     print('val: ', len(val_folds[f'fold{i}']), 'train: ', len(train_folds[f'fold{i}']), '\n')

In [12]:
# len(train_folds["fold0"]), len(train_files)

# for i_cv in range(n_splits):
#     print('Training classes\n')
#     print(np.unique([train_folds[f'fold{i_cv}'][i]['IDH_label'].item() for i in range(len(train_folds[f'fold{i_cv}']))], return_counts = True))
#     print('\nValidation classes\n')
#     print(np.unique([val_folds[f'fold{i_cv}'][i]['IDH_label'].item() for i in range(len(val_folds[f'fold{i_cv}']))], return_counts = True))
#     print('#'*4, '\n\n')

In [13]:
n_splits = 3
dfFolds = BraTS20SubjectsIDHTrainDCT
for i_cv in range(n_splits):
    print('Training classes\n')
    print(np.unique([BraTS20SubjectsIDHTrainDCT[f'fold{i_cv}'][i]['IDH_label'].item() for i in range(len(BraTS20SubjectsIDHTrainDCT[f'fold{i_cv}']))], return_counts = True))
    print('\nValidation classes\n')
    print(np.unique([BraTS20SubjectsIDHValDCT[f'fold{i_cv}'][i]['IDH_label'].item() for i in range(len(BraTS20SubjectsIDHValDCT[f'fold{i_cv}']))], return_counts = True))
    #print('#'*4, '\n')
    print('\nTesting classes\n')
    print(np.unique([BraTS20SubjectsIDHTestDCT[f'fold{i_cv}'][i]['IDH_label'].item() for i in range(len(BraTS20SubjectsIDHTestDCT[f'fold{i_cv}']))], return_counts = True))
    print('#'*40, '\n\n')

Training classes

(array([0., 1.]), array([79, 43]))

Validation classes

(array([0., 1.]), array([80, 42]))

Testing classes

(array([0., 1.]), array([79, 43]))
######################################## 


Training classes

(array([0., 1.]), array([80, 44]))

Validation classes

(array([0., 1.]), array([79, 43]))

Testing classes

(array([0., 1.]), array([79, 43]))
######################################## 


Training classes

(array([0., 1.]), array([79, 43]))

Validation classes

(array([0., 1.]), array([79, 43]))

Testing classes

(array([0., 1.]), array([80, 42]))
######################################## 




In [14]:
#train_folds['fold0'][2]

***HistogramStandardization***

Implementing histogram standardization from [torchIO](https://github.com/fepegar/torchio) library

Bases: [torchio.transforms.preprocessing.intensity.normalization_transform.NormalizationTransform](https://torchio.readthedocs.io/transforms/preprocessing.html#torchio.transforms.preprocessing.intensity.NormalizationTransform)

Perform histogram standardization of intensity values.

Implementation of [New variants of a method of MRI scale standardization](https://ieeexplore.ieee.org/document/836373).

We can visit in [torchio.transforms.HistogramStandardization.train()]((https://torchio.readthedocs.io/transforms/preprocessing.html#torchio.transforms.HistogramStandardization.train)) for more details.

PARAMETERS
landmarks – Dictionary (or path to a PyTorch file with .pt or .pth extension in which a dictionary has been saved) whose keys are image names in the subject and values are NumPy arrays or paths to NumPy arrays defining the landmarks after training with [torchio.transforms.HistogramStandardization.train()](https://torchio.readthedocs.io/transforms/preprocessing.html#torchio.transforms.HistogramStandardization.train).

Here, ***save_dir*** is a path where the trained histogram files for four channels (T1w, T1cw, T2w, and Flair), and trained model's weights will be saved

In [15]:
# file_prefix = 'ConvEffNet_Brats21_1SplitV0'
# savedirname = 'ConvEffNet_Brats21'
# save_dir = os.path.join('/raid/brats2021/pthBraTS2021Radiogenomics', savedirname)
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)

# train_images20T1 = dfTrainLbl['t1wPath'].values
# train_images20T1ce = dfTrainLbl['t1cwPath'].values
# train_images20T2 = dfTrainLbl['T2wPath'].values
# train_images20Flair = dfTrainLbl['FlairPath'].values

# hiseq_t1npyfile = os.path.join(save_dir, f"histeq_t1w_{file_prefix}.npy")
# t1w_landmarks = (hiseq_t1npyfile if os.path.isfile(hiseq_t1npyfile) else \
#                  tio.HistogramStandardization.train(train_images20T1, output_path = hiseq_t1npyfile))
# # #torch.save(t1w_landmarks, hiseq_t1npyfile)

# hiseq_t1cnpyfile =  os.path.join(save_dir, f"histeq_t1cw_{file_prefix}.npy")
# t1cw_landmarks = (hiseq_t1cnpyfile if os.path.isfile(hiseq_t1cnpyfile) else \
#                   tio.HistogramStandardization.train(train_images20T1ce, output_path = hiseq_t1cnpyfile))
# #torch.save(t1cw_landmarks, hiseq_t1cnpyfile)


# hiseq_t2npyfile = os.path.join(save_dir, f"histeq_t2w_{file_prefix}.npy")
# t2w_landmarks = (hiseq_t2npyfile if os.path.isfile(hiseq_t2npyfile) else \
#                  tio.HistogramStandardization.train(train_images20T2, output_path = hiseq_t2npyfile))
# #torch.save(t2w_landmarks, hiseq_t2npyfile)

# hiseq_flairnpyfile = os.path.join(save_dir, f"histeq_flair_{file_prefix}.npy")
# flair_landmarks = (hiseq_flairnpyfile if os.path.isfile(hiseq_flairnpyfile) else \
#                    tio.HistogramStandardization.train(train_images20Flair, output_path = hiseq_flairnpyfile))
# #torch.save(flair_landmarks, hiseq_flairnpyfile)


    
file_prefix = 'DyUNetPlus_BratsTCGA_1p19q_3CV_2Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Run2'
savedirname = 'DynUNetVariants_TCGA'
save_dir = os.path.join('/raid/brats2021/pthTCGA_1p19q_CoDeletion', savedirname)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

## Classes for Monai/Pytorch compose class

A class to rearrange label mask array as 
- [0, :, :, :] = the multi class mask (class labels: 0 (background), 1, 2, and 4)
- [1, :, :, :] = the whole tumor mask (class labels: 0 (background), and 1)\
Not using here

In [16]:
class ConvertToMultiChannelPlusWT(MapTransform):
    
    """
     GD-enhancing tumor (ET — label 4), 
     the peritumoral edema (ED — label 2), and 
     the necrotic and non-enhancing tumor core (NCR/NET — label 1)

    """
    
    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            result = []
            
            d[key]=np.squeeze(d[key], axis = 0) # Converting 1, H, W, D to H, W, D
            result.append(d[key])

            # merge labels 1, 2 and 4 to construct WT
            result.append(
                np.logical_or(
                    np.logical_or(d[key] == 2, d[key] == 4), d[key] == 1
                )
            )
            ## merge label 1 and label 4 to construct TC
            #result.append(np.logical_or(d[key] == 1, d[key] == 4))
            ## label 4 is ET
            #result.append(d[key] == 4)
            d[key] = np.stack(result, axis=0).astype(np.uint8)
        return d

#### Define a new transform to convert brain tumor labels
Here we convert the multi-classes labels into multi-labels segmentation task in One-Hot format.\
Not using here

In [17]:
class ConvertToMultiChannelBasedOnBratsClassesd(MapTransform):
    
    """
     GD-enhancing tumor (ET — label 4), 
     the peritumoral edema (ED — label 2), and 
     the necrotic and non-enhancing tumor core (NCR/NET — label 1)

    """
    
    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            result = []
            
            d[key]=np.squeeze(d[key], axis = 0) # Converting 1, H, W, D to H, W, D

            # merge labels 1, 2 and 4 to construct WT
            result.append(
                np.logical_or(
                    np.logical_or(d[key] == 2, d[key] == 4), d[key] == 1
                )
            )
            # merge label 1 and label 4 to construct TC
            result.append(np.logical_or(d[key] == 1, d[key] == 4))
            # label 4 is ET
            result.append(d[key] == 4)
            d[key] = np.stack(result, axis=0).astype(np.float32)
        return d

#### A class to add new key having the tumor mask (GT) to the existing data dictionary
The new key, ***label_mask*** will have the same dimension (size: 4,x,x,x) with image array (size: 4,x,x,x)\
Using in ***compose*** class

In [18]:
class ConvertToIDHLabel2WTd(MapTransform):
    
    """
     GD-enhancing tumor (ET — label 4), 
     the peritumoral edema (ED — label 2), and 
     the necrotic and non-enhancing tumor core (NCR/NET — label 1)

    """
    
    def __init__(self, keys: KeysCollection, IDH_label_key:str = 'IDH_label') -> None:

        super().__init__(keys)
        self.IDH_label_key = IDH_label_key
       
    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            # merge labels 1, 2 and 4 to construct WT
            #WT = np.logical_or(np.logical_or(d[key] == 2, d[key] == 4), d[key] == 1).astype(np.uint8)
            result = []
            WT = np.squeeze(d[key], axis = 0)
            if d[self.IDH_label_key].item() == 1:
                WT=np.multiply(WT, 2)
                #WT = 2*WT
            
            result.append(WT==1)
            result.append(WT==2)
            
            d[key] = np.stack(result, axis = 0).astype(np.float32)
        
        monai.data.utils.remove_keys(d, self.IDH_label_key)
            
    
        return d

In [19]:
class Convert2WTd(MapTransform):
    
    """
     GD-enhancing tumor (ET — label 4), 
     the peritumoral edema (ED — label 2), and 
     the necrotic and non-enhancing tumor core (NCR/NET — label 1)

    """
    
    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            result = []
            output_classes = 2
            
            # merge labels 1, 2 and 4 to construct WT
            WT = np.logical_or(np.logical_or(d[key] == 2, d[key] == 4), d[key] == 1).astype(np.float32)
            d[f'{key}'] = WT
         
            WT = np.expand_dims(ndimage.binary_dilation(np.squeeze(WT, axis=0), iterations=2), axis = 0).astype(np.float32)
            #WT = np.stack(tuple([ndimage.binary_dilation((np.squeeze(WT, axis = 0)==_k).astype(WT.dtype), iterations=5).astype(WT.dtype) for _k in range(output_classes)]), axis = 0)
            d[f'{key}_mask'] = WT
            d[f'{key}_mask_meta_dict'] = copy.deepcopy(d[f"{key}_meta_dict"])
            
        
        return d

In [20]:
class SpatialCropWTCOMd(MapTransform):
    
    """
     GD-enhancing tumor (ET — label 4), 
     the peritumoral edema (ED — label 2), and 
     the necrotic and non-enhancing tumor core (NCR/NET — label 1)

    """
    def __init__(self, keys: KeysCollection, roi_size, COM_label_key:str = 'label_mask') -> None:

        super().__init__(keys)
        self.COM_label_key = COM_label_key
        self.roi_size = roi_size
    
    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            result = []
            Coms = np.array([ndimage.measurements.center_of_mass(lbl) for lbl in list(d[self.COM_label_key])])
            Coms[np.isnan(Coms)] = 70
            Coms=Coms[0].astype(np.uint16).tolist()
        
            sc_com= SpatialCrop(roi_center= Coms, roi_size=self.roi_size)
            d[key] = sc_com(d[key])
                
        return d

In [21]:
class ConcatLabelBrainmaskd(MapTransform):
    """
          we do not need labels as it is a generative problem
    """
    
    def __init__(self, keys: KeysCollection, image_key = 'image', label_key = 'label', 
                 brain_mask_key = 'brain_mask') -> None:
        
        super().__init__(keys)
        self.brain_mask_key = brain_mask_key
        self.image_key = image_key
        self.label_key = label_key
    
    
    def __call__(self, data):
     
        d = dict(data)
        #d[self.image_key] = np.concatenate((d[self.image_key], d[self.label_key], d[self.brain_mask_key][0:1]), axis = 0)
        d[self.image_key] = np.concatenate((d[self.image_key], d[self.label_key][0:1]), axis = 0)
        
        return d

### Implementing channelwise histogram normalization
(Not using here)

In [22]:
class HistogramNormalizeChannelWised(MapTransform):
    """
          we do not need labels as it is a generative problem
    """
    
    def __init__(self, keys: KeysCollection, brain_mask_key = 'brain_mask', min=0, max=255) -> None:
        
        super().__init__(keys)
        self.brain_mask_key = brain_mask_key
        self.histnorms = HistogramNormalize(num_bins=256, min=min, max=max)
    
    
    def __call__(self, data):
     
        d = dict(data)
        for key in self.keys:
            nchnl = d[key].shape[0]
            for ch in range(nchnl):
                d[key][ch] = self.histnorms(d[key][ch], d[self.brain_mask_key][ch])
        
        return d

### Defining traning and validation transforms

- Training transform includes:
    - LoadImaged
    - EnsureChannelFirstd
    - HistogramNormalizeChannelWised: Histogram normalization channel wise (custom class defined aboove)
    - NormalizeIntensityd
    - RandRotate90d
    - RandZoomd
    - ConvertToIDHLabel2WTd (custom class defined above)
    - CropForegroundd: Cropping foreground based on the whole tumor mask (WT GT)
    - RandCropByPosNegLabeld: Randomly cropping 8 patches based on 3: 1 (WT : non tumor tissus) ratio
    - RandGaussianNoised
    - RandStdShiftIntensityd
    - RandFlipd
    
- validation transform includes:
    - LoadImaged
    - EnsureChannelFirstd
    - HistogramNormalizeChannelWised: Histogram normalization channel wise (custom class defined aboove)
    - NormalizeIntensityd
    - ConvertToIDHLabel2WTd (custom class defined above)
    
Most of transfroms are implemented using [Monai](https://docs.monai.io/en/latest/transforms.html#dictionary-transforms) library



In [23]:
def threshold_foreground(x):
    # threshold at not equal to 0
    return x == 1


#Resized(keys=keys[0:-1], spatial_size=patch_size, mode = ('area','nearest','nearest')),

# ConvertToMultiChannelBasedOnBratsClassesd(keys = ['label']),
# ConcatLabelBrainmaskd(keys = None, image_key = 'image', label_key = 'label', brain_mask_key = 'brain_mask'),
# CropForegroundd(keys=keys[0:-1], source_key="brain_mask", select_fn = threshold_foreground, start_coord_key='fg_start_coord', end_coord_key='fg_end_coord'),
# SpatialPadd(keys=keys[0:-1], spatial_size=patch_size),

# RandGaussianSmoothd(
#     keys=["image"],
#     sigma_x=(0.5, 1.15),
#     sigma_y=(0.5, 1.15),
#     sigma_z=(0.5, 1.15),
#     prob=0.3,
# ),

# RandScaleIntensityd(keys=["image"], factors=0.3, prob=0.3),
# RandGibbsNoised(keys=["image"], prob=0.3, alpha=(0.1, 0.5), as_tensor_output=False),

          
# DataStatsd(keys=keys[0:-1], prefix="Data", data_type=True, data_shape=True, value_range=True, data_value=False),
#DataStatsd(keys=keysExt[0:-1], prefix="Data", data_type=True, data_shape=True, value_range=True, data_value=False),


def get_task_transforms(patch_size, task='train', pos_sample_num=1, neg_sample_num=1, num_samples=1):
    
    #spatial_size=(30, 30, 30)
    orig_img_size = (240, 240, 155)

    if task=='train':
        keys = ["image", 'label', 'brain_mask', 'IDH_label']
        keysExt = ["image", 'label', 'brain_mask', 'label_mask', 'IDH_label']
        
        all_transform = [
            
            LoadImaged(keys=keys[0:-1], reader = "NibabelReader"),
            EnsureChannelFirstd(keys=keys[0:-1]),
            #adapter_tioChannelWise2monai(tiofn = tio.HistogramStandardization, mode = 'train',landmarks = landmarks_dict),
            #HistogramNormalizeChannelWised(keys = ['image'], brain_mask_key = 'brain_mask', min = 1, max = 65535),
            
            RandAffined(
                keys = keys[0:-1],
                prob=0.2,
                spatial_size= orig_img_size, #(240, 240, 155),
                rotate_range=np.pi/9,
                scale_range=(0.1, 0.1),
                mode=("bilinear", "nearest", "nearest"),
                as_tensor_output=False,
                padding_mode = ("zeros", "zeros", "zeros"),
            ),
            
            RandRotate90d(keys=keys[0:-1], prob=0.3, spatial_axes=[0, 2]),

            RandZoomd(
                keys=keys[0:-1],
                min_zoom=0.9,
                max_zoom=1.1,
                mode=("trilinear", "nearest", "nearest"),
                align_corners=(True, None, None),
                prob=0.3,
            ),
            
            #ConvertToIDHLabel2WTd(keys = ["label"]),
            CopyItemsd(keys=["label"], names=["label_mask"], times=1),
            Convert2WTd(keys = ["label"]),
            ConvertToIDHLabel2WTd(keys = ["label"], IDH_label_key = 'IDH_label'),
            CropForegroundd(keys=keysExt[0:-1], source_key="brain_mask", select_fn = threshold_foreground, start_coord_key='fg_start_coord', end_coord_key='fg_end_coord'),
            #Spacingd(keys = keysExt[0:-1], pixdim=(1.25, 1.25, 1.25), mode = ('bilinear','nearest', 'nearest', 'nearest')),
            NormalizeIntensityd(keys=["image"], nonzero=True, channel_wise=True),
            
            #ResizeWithPadOrCropd(keys = keysExt[0:-1], spatial_size = (128, 160, 128)),
            RandGaussianNoised(keys=["image"], std=0.01, prob=0.3),
            RandStdShiftIntensityd(keys = ["image"], factors=0.3, nonzero=True, channel_wise=True, prob=0.3), 
            RandFlipd(keys=keysExt[0:-1], prob=0.5, spatial_axis=0),
            RandFlipd(keys=keysExt[0:-1], prob=0.5, spatial_axis=1),
            RandFlipd(keys=keysExt[0:-1], prob=0.5, spatial_axis=2),
            
            CropForegroundd(keys=keysExt[0:-1], source_key="label_mask", select_fn = threshold_foreground, start_coord_key='fg_start_coord', end_coord_key='fg_end_coord', margin=2),
            #ResizeWithPadOrCropd(keys = keysExt[0:-1], spatial_size = patch_size),
            SpatialPadd(keys = keysExt[0:-1], spatial_size = patch_size),
#             RandCropByLabelClassesd(
#                 keys=keysExt[0:-1],            
#                 label_key = "label_mask",
#                 spatial_size = patch_size,    
#                 ratios= [1, 3],
#                 #ratios=[1,] * 2,
#                 num_classes=2,              
#                 num_samples=4,
#                 image_key="brain_mask",
#                 image_threshold=0.0,
#                 #allow_smaller = True,
#             ),
            
            RandCropByPosNegLabeld(
                keys=keysExt[0:-1],
                label_key="label_mask",
                spatial_size=patch_size,
                pos=pos_sample_num,
                neg=neg_sample_num,
                num_samples=num_samples,
                #image_key=None,
                image_key="brain_mask",
                image_threshold=0.,
            ),
            
            #SpatialCropWTCOMd(keys=keysExt[0:-1], roi_size=patch_size, COM_label_key = "label_mask"),
            SpatialPadd(keys = keysExt[0:-1], spatial_size = patch_size),
        
            #CastToTyped(keys=keysExt, dtype=(np.float32, np.uint8, np.uint8, np.uint8, np.float32)),
            CastToTyped(keys=keysExt[0:-1], dtype=(np.float32, np.float32, np.float32, np.float32)), #np.float32
            #ToTensord(keys=keysExt, dtype = (torch.float32, torch.float32, torch.float32, torch.float32, torch.float32)),
            #EnsureTyped(keys=keysExt[0:-1], data_type = "tensor", wrap_sequence=True),
#             #ToDeviced(keys = keys, device = deviceName),
        ]
        
        
        
    elif task=='validation':
    
        keys = ["image", 'label', 'brain_mask', 'IDH_label']
        keysExt = ["image", 'label', 'brain_mask', 'label_mask', 'IDH_label']
        all_transform = [
            
            LoadImaged(keys=keys[0:-1], reader = "NibabelReader"),
            EnsureChannelFirstd(keys=keys[0:-1]),
            #adapter_tioChannelWise2monai(tiofn = tio.HistogramStandardization, mode = 'train',landmarks = landmarks_dict),
            #HistogramNormalizeChannelWised(keys = ['image'], brain_mask_key = 'brain_mask', min = 1, max = 65535),
          
            CopyItemsd(keys=["label"], names=["label_mask"], times=1),
            Convert2WTd(keys = ["label"]),
            ConvertToIDHLabel2WTd(keys = ["label"], IDH_label_key = 'IDH_label'),
            CropForegroundd(keys=keysExt[0:-1], source_key="brain_mask", select_fn = threshold_foreground, start_coord_key='fg_start_coord', end_coord_key='fg_end_coord'),
            #Spacingd(keys = keysExt[0:-1], pixdim=(1.25, 1.25, 1.25), mode = ('bilinear','nearest', 'nearest', 'nearest')),
            NormalizeIntensityd(keys=["image"], nonzero=True, channel_wise=True),
            #ResizeWithPadOrCropd(keys = keysExt[0:-1], spatial_size = (128, 160, 128)),
            CropForegroundd(keys=keysExt[0:-1], source_key="label_mask", select_fn = threshold_foreground, start_coord_key='fg_start_coord', end_coord_key='fg_end_coord', margin=2),
            #SpatialCropWTCOMd(keys=keysExt[0:-1], roi_size=patch_size, COM_label_key = "label_mask"),
            SpatialPadd(keys = keysExt[0:-1], spatial_size = patch_size),
#             RandCropByPosNegLabeld(
#                 keys=keysExt[0:-1],
#                 label_key="label_mask",
#                 spatial_size=patch_size,
#                 pos=pos_sample_num,
#                 neg=neg_sample_num,
#                 num_samples=num_samples,
#                 image_key="brain_mask",
#                 image_threshold=0,
#             ),

            #CastToTyped(keys=keysExt, dtype=(np.float32, np.uint8, np.uint8, np.uint8, np.float32)),
            CastToTyped(keys=keysExt[0:-1], dtype=(np.float32, np.float32, np.float32, np.float32)), # np.float32
            #ToTensord(keys=keysExt, dtype = (torch.float32, torch.float32, torch.float32, torch.float32, torch.float32)),
            #ToTensord(keys=keysExt),
            #EnsureTyped(keys=keysExt[0:-1], data_type = "tensor", wrap_sequence=True),
            #ToDeviced(keys = keys, device = deviceName),
        ]
        
    else:
        print('print task either train or validation here')


    return Compose(all_transform)

# def create_cachedir(cache_dir):
#     if not os.path.exists(cache_dir):
#         os.makedirs(cache_dir)
#     return 1

### Section for visual inspection and debugging

In [24]:
#patch_size=(128, 160, 128)
#patch_size=(64, 80, 64)
patch_size=(32, 32, 32)
train_transforms = get_task_transforms(patch_size, task='train', pos_sample_num=3, neg_sample_num=1, num_samples=16)
val_transforms = get_task_transforms(patch_size, task='validation', pos_sample_num=1, neg_sample_num=1, num_samples=1)
len(train_transforms), len(val_transforms)

(20, 10)

In [25]:
all_train_dataset = monai.data.Dataset(data=BraTS20SubjectsIDHTrainDCT["fold0"], transform=train_transforms)
len(all_train_dataset)

122

In [26]:
#investifiles = copy.deepcopy(BraTS20SubjectsIDHTrainDCT["fold0"])

# for i in range(len(investifiles)):
#     investifiles[i]['IDH_label'] = investifiles[i]['IDH_label'].astype(np.float32) 
# all_train_dataset = monai.data.Dataset(data=investifiles, transform=train_transforms)
#all_train_dataset[10][3]['IDH_label']

In [27]:
# for cfold in tqdm(range(len(BraTS20SubjectsIDHTrainDCT))):
#     all_train_dataset = monai.data.Dataset(data=copy.deepcopy(BraTS20SubjectsIDHTrainDCT[f"fold{cfold}"]), transform=train_transforms)
#     dls = monai.data.DataLoader(all_train_dataset, batch_size=8, shuffle=False, collate_fn=list_data_collate)
#     # abatch = next(iter(dls))
#     # print(abatch['image'].shape)
#     # print(abatch['label'].shape)
#     for epoch in range(5):
#         for abatch in dls:
#             print(abatch['image'].shape)
#             print(abatch['label'].shape)
#             print(abatch['IDH_label'].shape)
#             print(abatch['IDH_label'], '\n', '###'*10, '\n')

In [28]:
#few_train_dataset = Dataset(data=train_files[0:10], transform=train_transforms)
# asub = few_train_dataset[5] 
# view(image = asub['image'][3].cpu(), label_image = asub['label_mask'][0].cpu())


### Few investigation

In [29]:
# afold_train_dataset = monai.data.Dataset(data=train_folds['fold0'], transform=train_transforms)
# #train_folds['fold0_IDH_label']
# uval, ucnt = np.unique(train_folds['fold0_IDH_label'], return_counts=True)
# weight = 1./(ucnt/ucnt.min())
# #weight = np.array([0.55, 0.45])
# sample_weights = np.array([weight[int(t)] for t in train_folds['fold0_IDH_label']])
# sample_weights = torch.from_numpy(sample_weights)
# weight, ucnt

In [30]:
#afold_train_dataset[200]['IDH_label'], afold_train_dataset[200]['label'].unique(return_counts = True)

## Custom editing of SegResNetVAE

In [31]:
def get_kernels_strides(sizes, spacings):
    #sizes, spacings = patch_size[task_id], spacing[task_id]
    strides, kernels = [], []

    while True:
        spacing_ratio = [sp / min(spacings) for sp in spacings]
        stride = [
            2 if ratio <= 2 and size >= 8 else 1
            for (ratio, size) in zip(spacing_ratio, sizes)
        ]
        kernel = [3 if ratio <= 2 else 1 for ratio in spacing_ratio]
        if all(s == 1 for s in stride):
            break
        sizes = [i / j for i, j in zip(sizes, stride)]
        spacings = [i * j for i, j in zip(spacings, stride)]
        kernels.append(kernel)
        strides.append(stride)
    strides.insert(0, len(spacings) * [1])
    kernels.append(len(spacings) * [3])
    return kernels, strides
#task_id = "01"
kernels, strides = get_kernels_strides(patch_size, spacing)
kernels.append([3, 3, 3])
strides.append([2, 2, 2])

print(kernels,'\n', strides)

print('strides length', len(strides))
#filters = [64, 96, 128, 192, 256, 384, 512, 768, 1024][: len(strides)]
#filters = [16, 32, 64, 128, 160, 160][: len(strides)]
#filters = [64, 128, 256, 384, 512, 768, 1024][: len(strides)]

filters = [32, 64, 128, 256, 320, 384, 512][: len(strides)]
print("Filters:", filters)

[[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]] 
 [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2]]
strides length 5
Filters: [32, 64, 128, 256, 320]


In [32]:
# kernels, strides = get_kernels_strides((128, 128, 128), spacing)
# #kernels, strides
# kernels = [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 6, 3]]
# strides = [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2]]

### Defining loss functions
- ***CrossEntropyLogitLoss*** Cross entropy logit loss from [PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)
- ***DiceCELoss*** Dice + Cross entropy loss from Monai
https://docs.monai.io/en/latest/losses.html


In [33]:
class CrossEntropyInstWLogitLoss(nn.Module):
    def __init__(self, is_smooth=False, label_smoothing = 0.1):
        super().__init__()
        self.is_smooth = is_smooth
        self.label_smoothing = label_smoothing
        
       
    def forward(self, y_pred, y_true):

        if self.is_smooth == True:
            y_true = y_true.float() * (1 - self.label_smoothing) + 0.5 * self.label_smoothing

        y_true=y_true.type_as(y_pred)   ### y_pred, and y_true should be same size and same data type
        
        
        #deviceidx = y_pred.get_device()
        #device = torch.device('cpu') if deviceidx == -1 else torch.device(f'cuda:{deviceidx}')
        #loss = F.binary_cross_entropy_with_logits(y_pred.to(device), y_true.to(device), pos_weight = weight.to(device))  ##pos_weight = weight 
        loss = F.binary_cross_entropy_with_logits(y_pred, y_true) 
        return loss


# class DeepDiceCELogitInstLoss(nn.Module):
#     def __init__(self):
#         super().__init__()
#         #self.volweight = torch.softmax(torch.tensor([0.12, 0.33, 0.55]), dim = 0)
#         self.dice = DiceLoss(include_background=False, to_onehot_y=True, softmax=True, squared_pred=True, batch = False)  # reduction = "none", batch = True
#         #self.smcross_entropy = CrossEntropyInstLoss()  ### was none torch.Tensor([0.66, 0.33, 1]), torch.tensor(self.volweight)

#     def forward(self, y_pred, y_true):
        
#         y_true = y_true.unsqueeze(dim=0).expand(y_pred.shape[1],-1,-1,-1,-1, -1)
#         #return sum([0.5 ** i * ((self.dice(p, l)) + self.smcross_entropy(p, l)) \
#         #            for i, (p, l) in enumerate(zip(torch.unbind(y_pred, dim=1), torch.unbind(y_true, dim=0)))])
#         return sum([0.5 ** i * self.dice(p, l) for i, (p, l) in enumerate(zip(torch.unbind(y_pred, dim=1), torch.unbind(y_true, dim=0)))])




class DeepDiceCELogitInstLoss(nn.Module):
    def __init__(self):
        super().__init__()
        #self.volweight = torch.softmax(torch.tensor([0.12, 0.33, 0.55]), dim = 0)
        self.dice = DiceLoss(to_onehot_y=False, sigmoid=True, squared_pred=True, batch = True, smooth_nr=0, smooth_dr=1e-5)  # reduction = "none", False
        self.logitcross_entropy = CrossEntropyInstWLogitLoss()  ### was none torch.Tensor([0.66, 0.33, 1]), torch.tensor(self.volweight)

    def forward(self, y_pred, y_true):
        
        y_true = y_true.unsqueeze(dim=0).expand(y_pred.shape[1],-1,-1,-1,-1, -1)
        return sum([0.5 ** i * ((self.dice(p, l)) + self.logitcross_entropy(p, l)) \
                    for i, (p, l) in enumerate(zip(torch.unbind(y_pred, dim=1), torch.unbind(y_true, dim=0)))])
    
loss_function = DeepDiceCELogitInstLoss()

In [34]:
xxp = torch.randint(0,4,size=(6,3, 128, 128, 128))
#pred = [torch.randn(6,3,8,8,6), torch.randn(6,3,8,8,6), torch.randn(6,3,8,8,6)]
pred = torch.stack([torch.randn(6, 3, 128, 128, 128), torch.randn(6, 3, 128, 128, 128), torch.randn(6, 3, 128, 128, 128), torch.randn(6, 3, 128, 128, 128)], dim=1)
loss_function(pred, xxp.float())
#loss_function(pred, xxp.float())

tensor(2.6450)

#### A function to create ***cache_dir*** to save transformed outputs

In [35]:
def removeAndcreate_cachedir(cache_dir):
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    else:
        #print("Pass")
        shutil.rmtree(cache_dir)
        os.makedirs(cache_dir)
    return 1

### Pytorch training loop

Following functionalities are added

- Implemeting learning rate finder
- Defining Ranger21 optimizer (learning rate scheduler attached to it)
- Implementing mixed precision (AMP)
- Saving the model weights based on the performance on validation data
- Executing 5 fold cross validation (CV) training/validation pipeline, saving a few best model's weights at each fold
- Defining train_dataset/train_loader, and val_dataset/val_loader at each fold
- Defining a CNN based classification model (DenseNet, EfficientNet, etc.) at each fold to make sure that all accumulated gradients get vanished

The key variables which are used here
- ***val_cache_dir:*** The path where the transformed ouputs of validaion files will be cached/saved
- ***train_cache_dir:*** The path where the transformed ouputs of training files will be cached/saved
- ***max_epochs:*** Total number of epochs
- ***save_dir:*** The path to save the checkpoints/weights of the model
- ***file_prefix:*** The text file where loss/accuracy is recoded like

```
current fold: 0 current epoch: 1, acc_metric: 0.4579 accuracy: 0.5085, f1score: 0.5085 epoch 1 average training loss: 0.7250 average validation loss: 0.7128 
current fold: 0 current epoch: 2, acc_metric: 0.4876 accuracy: 0.5424, f1score: 0.5424 epoch 2 average training loss: 0.6961 average validation loss: 0.6940 
current fold: 0 current epoch: 3, acc_metric: 0.4870 accuracy: 0.4915, f1score: 0.4915 epoch 3 average training loss: 0.6862 average validation loss: 0.6965 
current fold: 0 current epoch: 4, acc_metric: 0.4882 accuracy: 0.5593, f1score: 0.5593 epoch 4 average training loss: 0.6715 average validation loss: 0.6885 
current fold: 0 current epoch: 5, acc_metric: 0.5927 accuracy: 0.5593, f1score: 0.5593 epoch 5 average training loss: 0.6555 average validation loss: 0.6826 
```

- ***val_interval*** Epoch interval to investivate the model's performance on validation data. If the current performance is better than in previous epochs, the model's weights will be saved
- ***key_metric_n_saved*** The number of model checkpoints we want to save. It it is set as 5, top 5 checkpoints/weights will be saved in 5 different ***.pth*** files  


In [36]:

#***Executed pipeline***\
#<img src="assets/ProposedIDHClass.png" align="left" width="1024" height="1800">



In [37]:
def train(train_files, train_files_IDH_label, val_files, val_files_IDH_label, batch_size = 2, epochs = 10, find_lr=False, cfold = 0):
    

    #     model = DenseNet201(spatial_dims=2, in_channels=3,
    #                        out_channels=num_classes, pretrained=True).to(device)

    # create spatial 3D
    #model = MultiDenseNet(spatial_dims=3, in_channelsList=(4, 1, 1, 1, 1), out_channels=2, block_config = (6, 12, 24, 16)).to(device)
    #model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=4, out_channels=1).to(device)
    #model = monai.networks.nets.DenseNet264(spatial_dims=3, in_channels=4, out_channels=1, init_features=64, growth_rate=32, block_config=(6, 12, 64, 48)).to(device)
    #patch_size=(64, 80, 64)
    num_classes = 2
    
    model = DynUNet(
        spatial_dims=3,
        in_channels=2,
        out_channels=num_classes,
        kernel_size=kernels,
        strides=strides,
        upsample_kernel_size=strides[1:],
        norm_name="batch",
        filters = filters,
        #dropout=0.2,
        deep_supervision=True,
        res_block=True,
        deep_supr_num=2,
    ).to(device)
    
    
    
    
    auc_metric = ROCAUCMetric()
    

    #train_files, train_files_IDH_label, val_files, val_files_IDH_label = train_files[:48], train_files_IDH_label[:48], val_files[:16], val_files_IDH_label[:16]

    """
    Block for using Monai's caching mechanishm for faster training
    """
    
    file_prefixfold = file_prefix  ##or file_prefix f"{file_prefix}_fold{cfold}" if saving cv file
    data_rpath = '/home/mmiv-ml/data'
    train_cache_dir = os.path.join(data_rpath,f'cachingDataset/{file_prefixfold}/train')    
    val_cache_dir = os.path.join(data_rpath,f'cachingDataset/{file_prefixfold}/val')
    
    is_done_train = removeAndcreate_cachedir(train_cache_dir)
    is_done_val = removeAndcreate_cachedir(val_cache_dir)
    

    n_train_cache_n_trans = len(train_transforms) #15
    n_val_cache_n_trans = len(val_transforms)
    
     # create a training data loader

    train_dataset = monai.data.CacheNTransDataset(data=train_files, transform=train_transforms,\
                                               cache_n_trans = n_train_cache_n_trans, cache_dir = train_cache_dir)
    
    
    
    #train_dataset = monai.data.Dataset(data=train_files, transform=train_transforms)
    
#     #train_folds['fold0_IDH_label']
#     uval, ucnt = np.unique(train_files_IDH_label, return_counts=True)
#     weight = 1./(ucnt/ucnt.min())
#     #weight = 1./ucnt
#     #weight = np.array([0.55, 0.45])
#     sample_weights = np.array([weight[int(t)] for t in train_files_IDH_label])
#     sample_weights = torch.from_numpy(sample_weights)
#     sampler = WeightedRandomSampler(sample_weights, num_samples= len(sample_weights), replacement=True)


    #train_dataset = Dataset(data=train_files, transform=train_transforms)
    #train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2, pin_memory=True)
    #train_dataset = CacheDataset(data=train_files, transform=train_transforms, cache_rate = 1.0, num_workers=8)
    #train_loader = ThreadDataLoader(train_dataset, num_workers=0, batch_size=batch_size, shuffle=True)
    
    #shiffle = False, sampler = sampler, shuffle=True doesnot work with patch, num_workers=2
    train_loader = monai.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=list_data_collate) 
    

    
    # create a validation data loader
    
    #val_dataset = monai.data.Dataset(data=val_files, transform=val_transforms)
    #val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=2, pin_memory=True)
    #val_dataset = CacheDataset(data=val_files, transform=val_transforms, cache_rate = 1.0, num_workers=5)
    
    val_dataset = monai.data.CacheNTransDataset(data=val_files, transform=val_transforms,\
                                           cache_n_trans = n_val_cache_n_trans, cache_dir = val_cache_dir)
    #val_loader = ThreadDataLoader(val_dataset, num_workers=0, batch_size=1)
    val_loader = monai.data.DataLoader(val_dataset, batch_size=1, shuffle=False) #num_workers=2, pin_memory=True
    
    
    
#     for ibatch in train_loader:
#         ibatch_IDH = ibatch['IDH_label']
#         print(torch.eq(ibatch_IDH, 0).sum(), torch.eq(ibatch_IDH, 1).sum())
#         print('#'*50)
    
    
    """
    just initialising some basic steps
    """
    
    max_epochs = epochs
    find_lr=False
    
    ### Calling the loss function ***CrossEntropyPlusMSELoss**,and optimizer   
    #loss_function = nn.CrossEntropyLoss()
    #loss_function=nn.BCEWithLogitsLoss()
    
    #optimizer = torch.optim.Adam(model.parameters(), lr=1e-03, weight_decay=1e-07)
    optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-05, weight_decay = 1e-4)
    #optimizer = torch.optim.SGD(model.parameters(), lr=1e-03, momentum= 0.99, nesterov=True)
    #optimizer = torch.optim.Adam(model.parameters(), 1e-03, weight_decay=1e-04)
    scaler = torch.cuda.amp.GradScaler()
    
    
    max_lr_init = 1e-04
    """
     ###################### Block for LR finder from pytorch ignite ########################

    """

    if find_lr:

        def prepare_batch(batch, device=None, non_blocking=False):
            return _prepare_batch((batch['image'], batch['label'].long()), device, non_blocking)

        #trainer = create_supervised_trainer(model, optimizer, loss_function, device, non_blocking=False, prepare_batch=prepare_batch)
        def train_step(engine, batch):
            model.train()
            optimizer.zero_grad()
            x, y = batch['image'].to(device), batch['label'].to(device)  #non_blocking=True
            with torch.cuda.amp.autocast():
                y_pred = model(x)
                loss4lr = loss_function(y_pred, y)
                
            scaler.scale(loss4lr).backward()
            scaler.step(optimizer)
            scaler.update()
            return loss4lr.item()

        trainer = Engine(train_step)

        ProgressBar(persist=True).attach(trainer, output_transform=lambda x: {"batch loss": x})
        lr_finder = FastaiLRFinder()
        to_save={'model': model, 'optimizer': optimizer}
        num_iter = 100  #2*len(train_loader)
        run_epochs = int(np.ceil(num_iter/len(train_loader)))
        with lr_finder.attach(trainer, to_save, end_lr=1., num_iter=num_iter, diverge_th=1.5) as trainer_with_lr_finder:    ####diverge_th=1.5

            trainer_with_lr_finder.run(train_loader, max_epochs=run_epochs)  #max_epochs=run_epochs or 5

        ax = lr_finder.plot()
        plt.show()
        
        max_lr = lr_finder.lr_suggestion() if lr_finder.lr_suggestion()<1e-04 else max_lr_init
        #max_lr = lr_finder.lr_suggestion() ##max_lr/10 i guess not needed, ignite does itself
        print(f'Suggested learning rate by LR finder for this fold: {lr_finder.lr_suggestion()}')
        
    else:
        max_lr = max_lr_init
        
    #max_lr_slice = 1e01*max_lr if max_lr<5e-03 else 1e-02
    #max_lr_slice = 1e-01*max_lr if max_lr<1e-05 else max_lr
        
    
    """
    ### defining learning rate scheduler
    
    """
    #steps_per_epoch=len(train_loader)
    #optimizer.param_groups[0]['lr'] = max_lr #*1e-01
    #scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr_slice, steps_per_epoch=len(train_loader), epochs=max_epochs)
    #scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: (1 - epoch / max_epochs) ** 0.9)
    
    #max_lr = 1e-3   
    optimizer = Ranger21(model.parameters(), lr = max_lr, num_epochs = epochs, num_batches_per_epoch = len(train_loader))
    


    """
     ###################### Block for native pytorch training loop and  ########################
    """

    key_metric_n_saved = 2   ### Usually I keep it 5
    save_last = False 
    dispformat_specs = '.4f'

        
#     file_prefix = 'ConvEffNet_Brats21_5CV'
#     savedirname = 'ConvEffNet_Brats21'
#     save_dir = os.path.join('/raid/brats2021/pthBraTS2021Radiogenomics', savedirname)
#     if not os.path.exists(save_dir):
#         os.makedirs(save_dir)

    logsfile_path = f"{save_dir}/Logs_{file_prefix}.txt"


    epoch_num = max_epochs #  max_epochs
    val_interval = 1
    valstep = 0
    best_metric = -1
    best_metric_epoch = -1
    epoch_loss_values = list()
    metric_values = list()


    numsiters = len(train_files) // train_loader.batch_size

    #first_batch = monai.utils.misc.first(train_loader)
        
    
    #post_pred = AsDiscrete(argmax=True, to_onehot=num_classes)  ### num_classes=num_classes
    #post_label = AsDiscrete(to_onehot=num_classes) ###num_classes=num_classes
    #dice_metric = monai.metrics.DiceMetric(include_background=False, reduction='mean', get_not_nans=False)
    
    
    dice_metric = monai.metrics.DiceMetric(include_background=True, reduction='mean', get_not_nans=False)
    dice_metric_batch = monai.metrics.DiceMetric(include_background=True, reduction='mean_batch', get_not_nans=False)
    
    HD_metric = HausdorffDistanceMetric(include_background=True, percentile = 95., reduction='mean', get_not_nans=False)
    HD_metric_batch = HausdorffDistanceMetric(include_background=True, percentile = 95., reduction='mean_batch', get_not_nans=False)
    
    post_pred = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])  
    
    def one_hot_permute(x):
        return F.one_hot(x.squeeze(dim=0).long(), num_classes=num_classes).permute(3, 0, 1, 2)
    
    def get_binarize_tensor(x, dim=1):
        x_chlist = torch.unbind(x, dim = dim)
        bin_x = torch.zeros_like(x_chlist[0])
        for x_i in x_chlist:
            bin_x = torch.logical_or(x_i, bin_x)
        return bin_x.unsqueeze(dim=dim).to(torch.float32)
    
    def get_segclass(x, dim = 1):
        xdvc = x.device
        x_chlist = torch.unbind(x, dim = dim)
        xclassNoList = []
        xvalueList = []
        for x_i in x_chlist:
            
            xv, xc = torch.unique(x_i, return_counts  = True)

            if xc.shape[0]==1:
                xclassNoList.append(xc[0].item())
                xvalueList.append(xv[0].item())
            elif xc.shape[0]==2:
                    if torch.any(torch.eq(xv, 1)):
                        xclassNoList.append(xc[1].item())
                        xvalueList.append(xv[1].item())
                    else:
                        print('The function only supports binarized tensor (binarized unique values, 0(n=...) and 1(n=...) only)\n')
            else:
                print('The function only supports binarized tensor (binarized unique values, 0(n=...) and 1(n=...) only)\n')
        
        if torch.any(torch.eq(torch.tensor(xvalueList), 1)):  
            xclass = torch.argmax(torch.tensor(xclassNoList).to(xdvc))
        else:
            '''If all uniques class values are 0, we are assigning nan values as a class'''
            xclass = torch.tensor(float('NaN')).to(xdvc)
            
        return xclass 
            
#             xv, xc = torch.unique(x_i, return_counts  = True)
            
#             if torch.any(torch.eq(xv, 1)):
#                 xclassNoList.append(xc[1].item())
#             else:
#                 xclassNoList.append(0)        
#         xclass = torch.argmax(torch.tensor(xclassNoList).to(xdvc))          
       
                
    
    for epoch in range(epoch_num):
        print("-" * 10)
        print(f"epoch {epoch + 1}/{epoch_num}")
        model.train()
        epoch_loss = 0.
        stepiter = 0
        for batch_data in train_loader:
            
            stepiter += 1
            inputs, labels, IDH_labels= (
                batch_data['image'].to(device),
                batch_data['label'].to(device),
                batch_data['IDH_label'].to(device),
            )
            
          
            optimizer.zero_grad()
            
            with torch.cuda.amp.autocast():

                # compute output
                outputs  = model(inputs)
                loss = loss_function(outputs, labels) 

            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()

            print(f"{stepiter}/{numsiters}, train_loss: {loss.item():.4f}")

            #scheduler.step() 
            
        epoch_loss /= stepiter
        epoch_loss_values.append(epoch_loss)
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
        
        
        if (epoch + 1) % val_interval == 0:
            model.eval()
            with torch.no_grad():

                y_pred = torch.tensor([], dtype=torch.float32, device=device)
                y = torch.tensor([], dtype=torch.long, device=device)
                val_losses = torch.tensor([], dtype=torch.float32, device=device)


                for val_data in val_loader:

                    val_inputs, val_labels, val_IDH_labels = (
                        val_data['image'].to(device),
                        val_data['label'].to(device),
                        val_data['IDH_label'].to(device),
                    )
                
                    roi_size = patch_size #(32, 32, 32)
                    sw_batch_size = 8
                    val_overlap = 0.5
                    mode="gaussian"
                            
                    
                    with torch.cuda.amp.autocast():
                        
                        val_outputs = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model, mode = mode, overlap = val_overlap, sw_device = device, device = device) 
                        #val_outputs = model(val_inputs)
                        val_ce_loss = loss_function(val_outputs.unsqueeze(dim=1), val_labels)

                    val_losses = torch.cat([val_losses, val_ce_loss.view(1)], dim = 0)
                    val_outputs = torch.stack([post_pred(i) for i in torch.unbind(val_outputs, dim = 0)], dim = 0)
                    
                    
                    #val_labels2hot = torch.stack([one_hot_permute(i) for i in torch.unbind(val_labels, dim = 0)], dim = 0)

                    
                    val_labels_bin = get_binarize_tensor(val_labels, dim=1)
                    val_outputs_bin = get_binarize_tensor(val_outputs, dim=1)
                    
                    dice_metric(y_pred=val_outputs_bin, y=val_labels_bin)
                                        
                    klcc = KeepLargestConnectedComponent(applied_labels = [0, 1], is_onehot = True)  ##is_onehot=True or None by default
                    #val_labels = klcc(val_labels.squeeze(dim=0)).unsqueeze(dim=0)
                    val_labels = torch.stack([klcc(i) for i in torch.unbind(val_labels, dim = 0)], dim = 0)
                    
                    val_label4mSeg_C = get_segclass(val_outputs)
                    #val_surv_labels = val_surv_labels.squeeze(dim=1)  ###Squeezing from B, 1 to B if needed
                    y_pred = torch.cat([y_pred, val_label4mSeg_C.view(1)], dim=0)
                    y = torch.cat([y, val_IDH_labels], dim=0)

                mdice_value = dice_metric.aggregate()
                dice_metric.reset()
                
                y_pred, y = y_pred.cpu(), y.cpu()
                
                if torch.all(torch.isnan(y_pred))==True:
                    
                    auc_result, accscore, f1score = np.nan, np.nan, np.nan
                    #print('acc_metric#', np.nan, ', auc#', np.nan, ', f1#', np.nan, '\n')
                
                else:

                    num_nanvalues = torch.isnan(y_pred).sum().item()
                    not_nanmask = torch.logical_not(torch.isnan(y_pred))
                    y = y[not_nanmask]
                    y_pred = y_pred[not_nanmask]
                    
                    
                    acc_value = torch.eq(y_pred, y)
                    acc_metric = acc_value.sum().item() / len(acc_value)

                    '''auc metric'''
                    auc_metric(y_pred, y)
                    auc_result = auc_metric.aggregate()
                    auc_metric.reset()
                    
                    '''balanced accuracy and f1 score'''
                    accscore = balanced_accuracy_score(y, y_pred)
                    f1score = f1_score(y, y_pred, average='micro')
                    #print('acc_metric#', acc_metric, ', auc#', auc_result, ', f1#', f1score, '\n')

                del y, y_pred
            
            
                epoch_val_losses=torch.mean(val_losses).detach().cpu().item()
                #metric = auc_result
                mdice_value = mdice_value.item()
                #metric = mdice_value
                metric = (mdice_value+auc_result)/2
                metric= -1.0 if np.isnan(metric) else metric
                metric_values.append(metric) ######List of over number of epochs
                printstring = "Best AUC"
                

                with open(logsfile_path, 'a') as file:
                    file.write(
                        f"current fold: {cfold} current epoch: {epoch + 1} dice_score: {mdice_value:^{dispformat_specs}} acc_metric: {auc_result:^{dispformat_specs}}" 
                        f" accuracy: {accscore:^{dispformat_specs}}, f1score: {f1score:^{dispformat_specs}}"
                        f" epoch {epoch + 1} average training loss: {epoch_loss:^{dispformat_specs}} average validation loss: {epoch_val_losses:^{dispformat_specs}} \n"

                    )

                if valstep < key_metric_n_saved:
                    torch.save(model.state_dict(), os.path.join(save_dir, f"{file_prefix}_Fold{cfold}_{metric:^{dispformat_specs}}_epoch{epoch + 1}.pth"))
                    print(
                        f"current fold: {cfold} current epoch: {epoch + 1} dice_score: {mdice_value:^{dispformat_specs}} acc_metric: {auc_result:^{dispformat_specs}}" 
                        f" accuracy: {accscore:^{dispformat_specs}}, f1score: {f1score:^{dispformat_specs}}"
                        f" epoch {epoch + 1} average training loss: {epoch_loss:^{dispformat_specs}} average validation loss: {epoch_val_losses:^{dispformat_specs}}"
                        
                    )

                else:

                    #sortmetric_values = sorted(metric_values[:-1], reverse=True)  ###Higher loss needs to be deleted, so sorting is reversed
                    sortmetric_values = sorted(metric_values[:-1], reverse=False)  

                    if metric>=sortmetric_values[-key_metric_n_saved]:
                        savegood_metric = metric
                        good_metric_epoch = epoch + 1

                        #if os.path.exists(f"{save_dir}/{file_prefix}_{sortmetric_values[-key_metric_n_saved]:.4f}.pth"):
                        #    os.remove(f"{save_dir}/{file_prefix}_{sortmetric_values[-key_metric_n_saved]:.4f}.pth")
                        #else:
                        #    print("The file does not exist")

                        glblist = glob.glob(f"{save_dir}/{file_prefix}_Fold{cfold}_{sortmetric_values[-key_metric_n_saved]:^{dispformat_specs}}_*")

                        if not glblist:
                            print("The file does not exist")
                        else:
                            os.remove(glblist[0])


                        torch.save(model.state_dict(), os.path.join(save_dir, f"{file_prefix}_Fold{cfold}_{metric:^{dispformat_specs}}_epoch{epoch + 1}.pth"))
                        print("saved new best metric model")
                        print(
                            f"current fold: {cfold} current epoch: {epoch + 1} validation loss: {epoch_val_losses:^{dispformat_specs}}"
                            f" dice_score: {mdice_value:^{dispformat_specs}} acc_metric: {auc_result:^{dispformat_specs}}"
                            f" accuracy: {accscore:^{dispformat_specs}}, f1score: {f1score:^{dispformat_specs}}"
                            f"\n saved {printstring}: {savegood_metric:^{dispformat_specs}} at epoch: {good_metric_epoch}"
                        )

                    else:

                        f"current fold: {cfold} current epoch: {epoch + 1} validation loss: {epoch_val_losses:^{dispformat_specs}}"
                        f" dice_score: {mdice_value:^{dispformat_specs}} acc_metric: {auc_result:^{dispformat_specs}}"
                        f" accuracy: {accscore:^{dispformat_specs}}, f1score: {f1score:^{dispformat_specs}}"

                        #pass

                valstep += 1
        ####Saving last epoch
        if epoch==epoch_num-1:
            if save_last:
                torch.save(model.state_dict(), os.path.join(save_dir, f"{file_prefix}_Fold{cfold}_{metric:^{dispformat_specs}}_last_epoch{epoch + 1}.pth"))
            #break
            
    # Free up GPU memory after training
    model = None
    train_loader, val_loader = None, None
    gc.collect()
    torch.cuda.empty_cache()

### Loop to execute n_splits=3 fold cross validation
if the model is trained and the checkpoints are saved already, just setting the start_training flag as false, to run remaining part of the programs of this notebook

In [38]:
start_training = True


In [ ]:
n_splits = 3
if start_training:
    #### Running 10 folds
    for i in range(0, n_splits):
        
        #train_files_fld, train_files_fld_IDH_label, val_files_fld, val_files_fld_IDH_label  = copy.deepcopy(train_folds[f'fold{i}']), copy.deepcopy(train_folds[f'fold{i}_IDH_label']),\
        #copy.deepcopy(val_folds[f'fold{i}']), copy.deepcopy(val_folds[f'fold{i}_IDH_label'])
        
        
              
        train_files_fld, val_files_fld  = copy.deepcopy(BraTS20SubjectsIDHTrainDCT[f'fold{i}']), copy.deepcopy(BraTS20SubjectsIDHValDCT[f'fold{i}'])
        train_files_fld_IDH_label, val_files_fld_IDH_label = None, None
        batch_size=8
        ### Need to change batch size if minimux training batch size == 1
        print('fold', i, "Bacth Investigation, minimum batch size", len(train_files_fld)%batch_size)        
        #train(train_files_fld, train_files_fld_IDH_label, val_files_fld, val_files_fld_IDH_label, batch_size=batch_size, epochs = 200, cfold = i)
        #train(train_files_fld, train_files_fld_IDH_label, val_files_fld, val_files_fld_IDH_label, batch_size=batch_size, epochs = 500, cfold = i)
        train(train_files_fld, train_files_fld_IDH_label, val_files_fld, val_files_fld_IDH_label, batch_size=batch_size, epochs = 500, cfold = i)
        
        
    
    start_training = False
else:
    
    pass

fold 0 Bacth Investigation, minimum batch size 2
Ranger21 optimizer ready with following settings:

Core optimizer = AdamW
Learning rate of 0.0001

Important - num_epochs of training = ** 500 epochs **
please confirm this is correct or warmup and warmdown will be off

Warm-up: linear warmup, over 2000 iterations

Lookahead active, merging every 5 steps, with blend factor of 0.5
Norm Loss active, factor = 0.0001
Stable weight decay of 0.0001
Gradient Centralization = On

Adaptive Gradient Clipping = True
	clipping value of 0.01
	steps for clipping = 0.001

Warm-down: Linear warmdown, starting at 72.0%, iteration 5760 of 8000
warm down will decay until 3e-05 lr
----------
epoch 1/500


Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         239.00209204]


params size saved
total param groups = 1
total params in groups = 79
1/15, train_loss: 2.7223
2/15, train_loss: 2.6818
3/15, train_loss: 2.3777
4/15, train_loss: 2.4640
5/15, train_loss: 2.6853
6/15, train_loss: 2.6954
7/15, train_loss: 2.7039
8/15, train_loss: 2.2877
9/15, train_loss: 2.3632
10/15, train_loss: 2.3579
11/15, train_loss: 2.3577
12/15, train_loss: 2.3596
13/15, train_loss: 2.3822
14/15, train_loss: 2.4039
15/15, train_loss: 2.3669
16/15, train_loss: 2.2293
epoch 1 average loss: 2.4649
current fold: 0 current epoch: 1 dice_score: 0.2839 acc_metric: 0.5062 accuracy: 0.5062, f1score: 0.3525 epoch 1 average training loss: 2.4649 average validation loss: 1.5547
----------
epoch 2/500
1/15, train_loss: 2.7204
2/15, train_loss: 2.6797
3/15, train_loss: 2.3756
4/15, train_loss: 2.4604
5/15, train_loss: 2.6831
6/15, train_loss: 2.6924
7/15, train_loss: 2.7005
8/15, train_loss: 2.2844
9/15, train_loss: 2.3606
10/15, train_loss: 2.3549
11/15, train_loss: 2.3548
12/15, train_loss: 2

### Performing inference on dataset

In [38]:
def inferSubjectsWithTA(data_loader,listmodels, prediction_folder="./", topk=1, num_channels = 4,\
                orientation="LPS", withoptimizer = False, softmaxEnsemble=False, save_inference = False):
    """
    run inference, the output folder will be "./output"
    """        
    #device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    listmodels = listmodels[0:topk]
    for x in listmodels:
        print(f"available model file: {x}.")
        
    channel_nums =  monai.utils.misc.first(data_loader)['image'].shape[1] ##next(iter(val_loader["image"])).shape[1]
    channelNums = f"{channel_nums} channels"
    keys = ('image',)
    patch_size = (32, 32, 32)
    
    post_trans_sigbin = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
    post_trans_bin = Compose([AsDiscrete(threshold=0.5)])
    post_trans_sig = Compose([Activations(sigmoid=True)])
    
    auc_metric = ROCAUCMetric()
    dice_metric = monai.metrics.DiceMetric(include_background=True, reduction='mean', get_not_nans=False)
    dice_metric_batch = monai.metrics.DiceMetric(include_background=True, reduction='mean_batch', get_not_nans=False)
    
    HD_metric = HausdorffDistanceMetric(include_background=True, percentile = 95., reduction='mean', get_not_nans=False)
    HD_metric_batch = HausdorffDistanceMetric(include_background=True, percentile = 95., reduction='mean_batch', get_not_nans=False)
    
    post_pred = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])  
    
    def one_hot_permute(x):
        return F.one_hot(x.squeeze(dim=0).long(), num_classes=num_classes).permute(3, 0, 1, 2)
    
    def get_binarize_tensor(x, dim=1):
        x_chlist = torch.unbind(x, dim = dim)
        bin_x = torch.zeros_like(x_chlist[0])
        for x_i in x_chlist:
            bin_x = torch.logical_or(x_i, bin_x)
        return bin_x.unsqueeze(dim=dim).to(torch.float32)
        
    
    def get_segclass(x, dim = 1):
        xdvc = x.device
        x_chlist = torch.unbind(x, dim = dim)
        xclassNoList = []
        xvalueList = []
        for x_i in x_chlist:
            
            xv, xc = torch.unique(x_i, return_counts  = True)

            if xc.shape[0]==1:
                xclassNoList.append(xc[0].item())
                xvalueList.append(xv[0].item())
            elif xc.shape[0]==2:
                    if torch.any(torch.eq(xv, 1)):
                        xclassNoList.append(xc[1].item())
                        xvalueList.append(xv[1].item())
                    else:
                        print('The function only supports binarized tensor (binarized unique values, 0(n=...) and 1(n=...) only)\n')
            else:
                print('The function only supports binarized tensor (binarized unique values, 0(n=...) and 1(n=...) only)\n')
        
        if torch.any(torch.eq(torch.tensor(xvalueList), 1)):  
            xclass = torch.argmax(torch.tensor(xclassNoList).to(xdvc))
        else:
            '''If all uniques class values are 0, we are assigning nan values as a class'''
            xclass = torch.tensor(float('NaN')).to(xdvc)
            
        return xclass 
    
    
    
    keys = ("image",)
        
    with torch.no_grad():
        
        y_pred = torch.tensor([], dtype=torch.float32, device=device)
        y = torch.tensor([], dtype=torch.long, device=device)
        Infer_idLst = list()
    
        for infindx, infer_data in enumerate(tqdm(data_loader)):

            
            val_inputs, val_labels, val_IDH_labels = (
                infer_data['image'].to(device),
                infer_data['label'].to(device),
                infer_data['IDH_label'].to(device),
            )
                

            n_class = 2
            val_outputsAll = torch.zeros(val_inputs.shape[0], n_class, val_inputs.shape[2], val_inputs.shape[3], val_inputs.shape[4]).to(device)
            n_model = 0.
            
            #for mdlindx in (number+1 for number in range(topk)):
            for mdlindx in range(topk):
    
                print(f'Model {mdlindx}, {listmodels[mdlindx]} is running now')
                model = None          
                model = DynUNet(
                    spatial_dims=3,
                    in_channels=4,
                    out_channels=n_class,
                    kernel_size=kernels,
                    strides=strides,
                    upsample_kernel_size=strides[1:],
                    norm_name="batch",
                    filters = filters,
                    deep_supervision=True,
                    res_block=True,
                    deep_supr_num=2,
                ).to(device)
                
                
                
                
                if withoptimizer ==True:
                    
                    state_dictsAll = torch.load(listmodels[mdlindx], map_location=device)
                    model.load_state_dict(state_dictsAll["model_state_dict"])
                    model.eval()
                
                else:    
                    model.load_state_dict(torch.load(listmodels[mdlindx], map_location=device))
                    model.eval()
                
                
                n = 1.0
                roi_size = patch_size #(32, 32, 32)
                sw_batch_size = 8
                val_overlap = 0.5
                mode="gaussian"
                
                with torch.cuda.amp.autocast():

                    preds = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model, mode = mode, overlap = val_overlap, sw_device = device, device = device)
                    
                flip_val_inputs = torch.flip(val_inputs, dims=(2, 3, 4))
                
                with torch.cuda.amp.autocast():
                    
                    mfpred = sliding_window_inference(flip_val_inputs, roi_size, sw_batch_size, model, mode = mode, overlap = val_overlap, sw_device = device, device = device)
                 
                flip_pred = torch.flip(mfpred, dims=(2, 3, 4))
                preds  = preds + flip_pred
                n = n + 1.0
                    
                
                for _ in range(4):
                    # test time augmentations
                    _img = RandGaussianNoised(keys[0], prob=1.0, std=0.01)(infer_data)[keys[0]]


                    with torch.cuda.amp.autocast():

                        #val_outputs = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model, sw_device = device, device = device)
                        _img_pred = sliding_window_inference(_img.to(device), roi_size, sw_batch_size, model, mode = mode, overlap = val_overlap, sw_device = device, device = device)
                        preds = preds + _img_pred
                        n = n + 1.0

                    
                    _img_flip = torch.flip(_img, dims=(2, 3, 4)) 
                    with torch.cuda.amp.autocast():
                        _mf_flip_pred = sliding_window_inference(_img_flip.to(device), roi_size, sw_batch_size, model, mode = mode, overlap = val_overlap, sw_device = device, device = device)
                    
                    _img_flip_pred = torch.flip(_mf_flip_pred, dims=(2, 3, 4))
                    preds = preds + _img_flip_pred
                    n = n + 1.0
                 
                
                preds = preds / n
                
                if softmaxEnsemble:
                    preds = torch.stack([post_trans_sig(i) for i in torch.unbind(preds, dim = 0)], dim = 0)
                val_outputsAll = val_outputsAll + preds
                n_model = n_model+1.0
                
                # Free up GPU memory after training
                model = None
                del model
                #train_loader, val_loader = None, None        
                gc.collect()
                torch.cuda.empty_cache()
                           
            val_outputsAll = val_outputsAll / n_model
            
            val_outputs = post_trans_bin(val_outputsAll) if softmaxEnsemble else post_trans_sigbin(val_outputsAll)
            '''Sigmoid or logit'''
            val_outputsSig = val_outputsAll if softmaxEnsemble else post_trans_sig(val_outputsAll)
            #val_outputsSig = val_outputsAll
            
            

            #val_labels2hot = torch.stack([one_hot_permute(i) for i in torch.unbind(val_labels, dim = 0)], dim = 0)


            val_labels_bin = get_binarize_tensor(val_labels, dim=1)
            val_outputs_bin = get_binarize_tensor(val_outputs, dim=1)

            dice_metric(y_pred=val_outputs_bin, y=val_labels_bin)
            HD_metric(y_pred=val_outputs_bin, y=val_labels_bin)

            klcc = KeepLargestConnectedComponent(applied_labels = [0, 1], is_onehot = True)  ##is_onehot=True or None by default
            #val_labels = klcc(val_labels.squeeze(dim=0)).unsqueeze(dim=0)
            val_labels = torch.stack([klcc(i) for i in torch.unbind(val_labels, dim = 0)], dim = 0)

            val_label4mSeg_C = get_segclass(val_outputs)
            #val_surv_labels = val_surv_labels.squeeze(dim=1)  ###Squeezing from B, 1 to B if needed
            y_pred = torch.cat([y_pred, val_label4mSeg_C.view(1)], dim=0)
            y = torch.cat([y, val_IDH_labels], dim=0)
                
            
            if save_inference:  
            
                val_outputs4save = val_outputs.squeeze(dim = 0)
                '''Sigmoid or logit'''
                val_outputsSig4Save = val_outputsSig.squeeze(dim = 0)


                ##### For padding going back to original position

                val_outputsPadded = np.zeros(shape = (val_outputs4save.shape[0], 240, 240, 155))
                '''Sigmoid or logit'''
                val_outputsSigPadded = np.zeros(shape = (val_outputsSig4Save.shape[0], 240, 240, 155))


                infr_start_cord = infer_data["fg_start_coord"][0]
                infr_end_cord = infer_data["fg_end_coord"][0]
                start_coord = infr_start_cord[0].item(), infr_start_cord[1].item(), infr_start_cord[2].item() 
                end_coord = infr_end_cord[0].item(), infr_end_cord[1].item(), infr_end_cord[2].item()

                if orientation =="LPS":

                    val_outputs_now_affine = infer_data['image_meta_dict']['affine'][0]
                    val_outputsLPSData = val_outputs4save.cpu().numpy()
                    '''Sigmoid or logit'''
                    val_outputsSigLPSData = val_outputsSig4Save.cpu().numpy()


                else:
                    print("Stopp the execution")


                xs, xe = 0,val_outputsLPSData.shape[1]
                ys, ye = 0,val_outputsLPSData.shape[2]
                zs, ze = 0,val_outputsLPSData.shape[3]

                if((end_coord[0]-start_coord[0])<roi_size[0]):
                    padlen = roi_size[0]-(end_coord[0]-start_coord[0])
                    xs = padlen//2
                    xe = roi_size[0] -(padlen - xs)
                if((end_coord[1]-start_coord[1])<roi_size[1]):
                    padlen = roi_size[1]-(end_coord[1]-start_coord[1])
                    ys = padlen//2
                    ye = roi_size[1] -(padlen - ys)
                if((end_coord[2]-start_coord[2])<roi_size[2]):
                    padlen = roi_size[2]-(end_coord[2]-start_coord[2])
                    zs = padlen//2
                    ze = roi_size[2] - (padlen - zs)

                val_outputsPadded[:, start_coord[0]:end_coord[0], start_coord[1]:end_coord[1], start_coord[2]:end_coord[2]] = \
                val_outputsLPSData[:, xs:xe, ys:ye, zs:ze]

                '''Sigmoid or logit'''
                val_outputsSigPadded[:, start_coord[0]:end_coord[0], start_coord[1]:end_coord[1], start_coord[2]:end_coord[2]] = \
                val_outputsSigLPSData[:, xs:xe, ys:ye, zs:ze]


                ''' If both classes (IDH and nonIDH) overlaps, assigning to the region to non-IDH'''

                WT_noidh = torch.from_numpy(val_outputsPadded[0:1,:, :, :]).long()
                WT_idh = torch.from_numpy(val_outputsPadded[1:2,:, :, :]).long()

                WT=torch.logical_or(WT_noidh, WT_idh).long()

                WT_noidhMask = torch.eq(WT_noidh,1)
                WTidhOnly = WT.masked_fill(WT_noidhMask, 0)
                WTidhOnly2=torch.where(WTidhOnly==1, 2, Edema)

                mergedWT=WTidhOnly2 + WT_noidh


                print('#'*20)
                bi = 0
                input_file_path = infer_data['image_meta_dict']['filename_or_obj'][bi]
                subID=os.path.basename(os.path.dirname(input_file_path))
                print('Subject ID: ', subID)



                val_outputsPadded_3class=WTidhOnly2 + WT_noidh #(0=background, 1=no IDH, 2= IDH)
                val_outputsPadded_3class = torch.where(val_outputsPadded_3class==3, 1, val_outputsPadded_3class).numpy().astype(np.uint8)

                val_outputsPadded_3class = np.moveaxis(val_outputsPadded_3class, 0, -1).astype(np.float32)
                val_outputsSigPadded = np.moveaxis(val_outputsSigPadded, 0, -1).astype(np.float32)

                #input_file_path = infer_data['image_meta_dict']['filename_or_obj'][bi]
                #subID=os.path.basename(os.path.dirname(input_file_path))

                if channel_nums==4:
                    new_file_name_sig = subID + "_sigmoid.nii.gz"
                    new_file_name = subID + ".nii.gz"
                elif channel_nums==2:
                    new_file_name_sig = subID + "_sigmoid.nii.gz"
                    new_file_name = subID + ".nii.gz"
                else:
                    print("The program is not saving")


                if not os.path.exists(prediction_folder):
                    os.makedirs(prediction_folder)

                prediction_folder_sub = os.path.join(prediction_folder, subID)
                if not os.path.exists(prediction_folder_sub):
                    os.makedirs(prediction_folder_sub)


                exact_file_path = os.path.join(prediction_folder_sub, new_file_name)
                exact_file_sig_path = os.path.join(prediction_folder_sub, new_file_name_sig)

                affine = infer_data['image_meta_dict']['affine'][bi]
                target_affine = infer_data['image_meta_dict']['original_affine'][bi]
                #affine=val_outputs_now_affine
                #target_affine=val_outputs_now_affine

                resample=False
                mode = 'nearest'
                monai.data.write_nifti(data=val_outputsPadded_3class, file_name=exact_file_path, affine=affine, target_affine=target_affine, resample=resample)
                monai.data.write_nifti(data=val_outputsSigPadded, file_name=exact_file_sig_path, affine=affine, target_affine=target_affine, resample=resample)

                
    
                ## Free up GPU memory after training
                #model = None
                #del model
                ##train_loader, val_loader = None, None        
                #gc.collect()
                #torch.cuda.empty_cache()
                
            
            
         
            WTdices.append(dice_metric.aggregate().item())
            dice_metric.reset()
            
            WTHD95s.append(HD_metric.aggregate().item())
            HD_metric.reset()

            input_file_path = infer_data['image_meta_dict']['filename_or_obj'][bi]
            subID=os.path.basename(os.path.dirname(input_file_path))
            print('Subject ID: ', subID)
            Infer_idLst.append(subID)

        y_pred, y = y_pred.cpu(), y.cpu()
        
        
        
        aDF = pd.DataFrame.from_dict({"SubjectID": Infer_idLst,  "Model": ["DynUnet"]* len(WTdices), "Channels":[channelNums]*len(WTdices), "Tumor regions": ["WT"]*len(WTdices), "Dice score": WTdices, "HD95": WTHD95s,\
                                     'ylabel':y.tolist(), 'ypred': y_pred.tolist()})
        del y, y_pred
        
                    
#     dfET = pd.DataFrame({"BraTS21ID":Infer_idLst, "Model": ["DynUnet"]* len(ETdices), "Channels":[channelNums]*len(ETdices), "Tumor regions": ["ET"]*len(ETdices), "Dice score": ETdices, "HD95": ETHD95s})
#     dfTC = pd.DataFrame({"BraTS21ID":Infer_idLst, "Model": ["DynUnet"]* len(TCdices), "Channels":[channelNums]*len(TCdices), "Tumor regions": ["TC"]*len(TCdices), "Dice score": TCdices, "HD95": TCHD95s})
#     dfWT = pd.DataFrame({"BraTS21ID":Infer_idLst,"Model": ["DynUnet"]* len(WTdices), "Channels":[channelNums]*len(WTdices), "Tumor regions": ["WT"]*len(WTdices), "Dice score": WTdices, "HD95": WTHD95s})
#     dfWT_TC_ET = pd.DataFrame({"BraTS21ID":Infer_idLst,"Model": ["DynUnet"]* len(WTdices), "Channels":[channelNums]*len(WTdices), "Tumor regions": ["WT_TC_ET_Regions"]*len(WTdices), "Dice score": Alldices, "HD95": AllHD95s})
    
#     dfRegions = pd.concat([dfTC, dfWT, dfET, dfWT_TC_ET])
#     return dfRegions


    return aDF

    
            

In [39]:
def inferWithTA(data_loader,listmodels, prediction_folder="./", topk=1, num_channels = 4,\
                orientation="LPS", withoptimizer = False, softmaxEnsemble=False, save_inference = False, tta = False):
    """
    run inference, the output folder will be "./output"
    """        
    #device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    listmodels = listmodels[0:topk]
    for x in listmodels:
        print(f"available model file: {x}.")
        
    channel_nums =  monai.utils.misc.first(data_loader)['image'].shape[1] ##next(iter(val_loader["image"])).shape[1]
    channelNums = f"{channel_nums} channels"
    keys = ('image',)
    patch_size = (32, 32, 32)
    
    post_trans_sigbin = Compose([EnsureType(), Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
    post_trans_bin = Compose([EnsureType(), AsDiscrete(threshold=0.5)])
    post_trans_sig = Compose([EnsureType(), Activations(sigmoid=True)])
    
    auc_metric = ROCAUCMetric()
    dice_metric = monai.metrics.DiceMetric(include_background=True, reduction='mean', get_not_nans=False)
    dice_metric_batch = monai.metrics.DiceMetric(include_background=True, reduction='mean_batch', get_not_nans=False)
    
    
    HD_metric = HausdorffDistanceMetric(include_background=True, percentile = 95., reduction='mean', get_not_nans=False)
    HD_metric_batch = HausdorffDistanceMetric(include_background=True, percentile = 95., reduction='mean_batch', get_not_nans=False)
    
    
    
    post_pred = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])  
    
    def one_hot_permute(x):
        return F.one_hot(x.squeeze(dim=0).long(), num_classes=num_classes).permute(3, 0, 1, 2)
    
    def get_binarize_tensor(x, dim=1):
        x_chlist = torch.unbind(x, dim = dim)
        bin_x = torch.zeros_like(x_chlist[0])
        for x_i in x_chlist:
            bin_x = torch.logical_or(x_i, bin_x)
        return bin_x.unsqueeze(dim=dim).to(torch.float32)
        
    
    def get_segclass(x, dim = 1):
        xdvc = x.device
        x_chlist = torch.unbind(x, dim = dim)
        xclassNoList = []
        xvalueList = []
        for x_i in x_chlist:
            
            xv, xc = torch.unique(x_i, return_counts  = True)

            if xc.shape[0]==1:
                xclassNoList.append(xc[0].item())
                xvalueList.append(xv[0].item())
            elif xc.shape[0]==2:
                    if torch.any(torch.eq(xv, 1)):
                        xclassNoList.append(xc[1].item())
                        xvalueList.append(xv[1].item())
                    else:
                        print('The function only supports binarized tensor (binarized unique values, 0(n=...) and 1(n=...) only)\n')
            else:
                print('The function only supports binarized tensor (binarized unique values, 0(n=...) and 1(n=...) only)\n')
        
        if torch.any(torch.eq(torch.tensor(xvalueList), 1)):  
            xclass = torch.argmax(torch.tensor(xclassNoList).to(xdvc))
        else:
            '''If all uniques class values are 0, we are assigning nan values as a class'''
            xclass = torch.tensor(float('NaN')).to(xdvc)
            
        return xclass 
    
    
    
    keys = ("image",)
        
    with torch.no_grad():
        
        y_pred = torch.tensor([], dtype=torch.float32, device=device)
        y = torch.tensor([], dtype=torch.long, device=device)
    
        for infindx, infer_data in enumerate(tqdm(data_loader)):

            
            val_inputs, val_labels, val_IDH_labels = (
                infer_data['image'].to(device),
                infer_data['label'].to(device),
                infer_data['IDH_label'].to(device),
            )
                

            n_class = 2
            val_outputsAll = torch.zeros(val_inputs.shape[0], n_class, val_inputs.shape[2], val_inputs.shape[3], val_inputs.shape[4]).to(device)
            n_model = 0.
            
            #for mdlindx in (number+1 for number in range(topk)):
            for mdlindx in range(topk):
    
                print(f'Model {mdlindx}, {listmodels[mdlindx]} is running now')
                model = None          
                model = DynUNet(
                    spatial_dims=3,
                    in_channels=4,
                    out_channels=n_class,
                    kernel_size=kernels,
                    strides=strides,
                    upsample_kernel_size=strides[1:],
                    norm_name="batch",
                    filters = filters,
                    deep_supervision=True,
                    res_block=True,
                    deep_supr_num=2,
                ).to(device)
                
                
                
                
                if withoptimizer ==True:
                    
                    state_dictsAll = torch.load(listmodels[mdlindx], map_location=device)
                    model.load_state_dict(state_dictsAll["model_state_dict"])
                    model.eval()
                
                else:    
                    model.load_state_dict(torch.load(listmodels[mdlindx], map_location=device))
                    model.eval()
                
                
                n = 1.0
                roi_size = patch_size #(32, 32, 32)
                sw_batch_size = 8
                val_overlap = 0.5
                mode="gaussian"
                
                with torch.cuda.amp.autocast():

                    preds = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model, mode = mode, overlap = val_overlap, sw_device = device, device = device)
                    
                flip_val_inputs = torch.flip(val_inputs, dims=(2, 3, 4))
                
                with torch.cuda.amp.autocast():
                    
                    mfpred = sliding_window_inference(flip_val_inputs, roi_size, sw_batch_size, model, mode = mode, overlap = val_overlap, sw_device = device, device = device)
                 
                flip_pred = torch.flip(mfpred, dims=(2, 3, 4))
                preds  = preds + flip_pred
                n = n + 1.0
                
                if tta:
                    
                    for _ in range(4):
                        # test time augmentations
                        _img = RandGaussianNoised(keys[0], prob=1.0, std=0.01)(infer_data)[keys[0]]


                        with torch.cuda.amp.autocast():

                            #val_outputs = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model, sw_device = device, device = device)
                            _img_pred = sliding_window_inference(_img.to(device), roi_size, sw_batch_size, model, mode = mode, overlap = val_overlap, sw_device = device, device = device)
                            preds = preds + _img_pred
                            n = n + 1.0


                        _img_flip = torch.flip(_img, dims=(2, 3, 4)) 
                        with torch.cuda.amp.autocast():
                            _mf_flip_pred = sliding_window_inference(_img_flip.to(device), roi_size, sw_batch_size, model, mode = mode, overlap = val_overlap, sw_device = device, device = device)

                        _img_flip_pred = torch.flip(_mf_flip_pred, dims=(2, 3, 4))
                        preds = preds + _img_flip_pred
                        n = n + 1.0
                 
                
                preds = preds / n
                
                if softmaxEnsemble:
                    preds = torch.stack([post_trans_sig(i) for i in torch.unbind(preds, dim = 0)], dim = 0)
                val_outputsAll = val_outputsAll + preds
                n_model = n_model+1.0
                
                # Free up GPU memory after training
                model = None
                del model
                #train_loader, val_loader = None, None        
                gc.collect()
                torch.cuda.empty_cache()
                           
            val_outputsAll = val_outputsAll / n_model
            
            val_outputs = post_trans_bin(val_outputsAll) if softmaxEnsemble else post_trans_sigbin(val_outputsAll)
            '''Sigmoid or logit'''
            val_outputsSig = val_outputsAll if softmaxEnsemble else post_trans_sig(val_outputsAll)
            #val_outputsSig = val_outputsAll
            
            

            #val_labels2hot = torch.stack([one_hot_permute(i) for i in torch.unbind(val_labels, dim = 0)], dim = 0)


            val_labels_bin = get_binarize_tensor(val_labels, dim=1)
            val_outputs_bin = get_binarize_tensor(val_outputs, dim=1)

            dice_metric(y_pred=val_outputs_bin, y=val_labels_bin)

            klcc = KeepLargestConnectedComponent(applied_labels = [0, 1], is_onehot = True)  ##is_onehot=True or None by default
            #val_labels = klcc(val_labels.squeeze(dim=0)).unsqueeze(dim=0)
            val_labels = torch.stack([klcc(i) for i in torch.unbind(val_labels, dim = 0)], dim = 0)

            val_label4mSeg_C = get_segclass(val_outputs)
            #val_surv_labels = val_surv_labels.squeeze(dim=1)  ###Squeezing from B, 1 to B if needed
            y_pred = torch.cat([y_pred, val_label4mSeg_C.view(1)], dim=0)
            y = torch.cat([y, val_IDH_labels], dim=0)
                
                        
            
        mdice_value = dice_metric.aggregate().item()
        dice_metric.reset()

        y_pred, y = y_pred.cpu(), y.cpu()
        
        num_nanvalues = torch.isnan(y_pred).sum().item()
        not_nanmask = torch.logical_not(torch.isnan(y_pred))
        y = y[not_nanmask]
        y_pred = y_pred[not_nanmask]
        
        
        acc_value = torch.eq(y_pred, y)
        acc_metric = acc_value.sum().item() / len(acc_value)

        #y_onehot = [post_label(i) for i in torch.unbind(y, dim=0)]
        #y_pred_act = [post_pred(i) for i in torch.unbind(y_pred, dim=0)]

        auc_metric(y_pred, y)
        auc_result = auc_metric.aggregate()
        auc_metric.reset()

        accscore = balanced_accuracy_score(y, y_pred)
        f1score = f1_score(y, y_pred, average='micro')
        del y, y_pred
        
        aDCT = {"dice_score": mdice_value,  'acc_metric':auc_result,  "accuracy": accscore,  'f1score':f1score, 'NanSubjectNos':num_nanvalues}
        
                    
#     dfET = pd.DataFrame({"BraTS21ID":Infer_idLst, "Model": ["DynUnet"]* len(ETdices), "Channels":[channelNums]*len(ETdices), "Tumor regions": ["ET"]*len(ETdices), "Dice score": ETdices, "HD95": ETHD95s})
#     dfTC = pd.DataFrame({"BraTS21ID":Infer_idLst, "Model": ["DynUnet"]* len(TCdices), "Channels":[channelNums]*len(TCdices), "Tumor regions": ["TC"]*len(TCdices), "Dice score": TCdices, "HD95": TCHD95s})
#     dfWT = pd.DataFrame({"BraTS21ID":Infer_idLst,"Model": ["DynUnet"]* len(WTdices), "Channels":[channelNums]*len(WTdices), "Tumor regions": ["WT"]*len(WTdices), "Dice score": WTdices, "HD95": WTHD95s})
#     dfWT_TC_ET = pd.DataFrame({"BraTS21ID":Infer_idLst,"Model": ["DynUnet"]* len(WTdices), "Channels":[channelNums]*len(WTdices), "Tumor regions": ["WT_TC_ET_Regions"]*len(WTdices), "Dice score": Alldices, "HD95": AllHD95s})
    
#     dfRegions = pd.concat([dfTC, dfWT, dfET, dfWT_TC_ET])
#     return dfRegions


    return aDCT

    
            

In [40]:
# listmodels = glob.glob(f'{save_dir}/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_1PatchEp500V2_Fold0_0.8371_epoch207.pt*') 
# listmodels.extend(glob.glob(f'{save_dir}/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_1PatchEp500V2_Fold1_0.8944_epoch75.pt*')) 
# listmodels.extend(glob.glob(f'{save_dir}/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_1PatchEp500V2_Fold2_0.8719_epoch482.pt*'))
# print(listmodels)

prediction_folder = f'{save_dir}/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_1PatchEp500V2' 
modelDCTList = {'fold0': [glob.glob(f'{save_dir}/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold0_0.8423_epoch197.pt*')[0],
                          glob.glob(f'{save_dir}/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold0_0.8422_epoch115.pth*')[0]],\
               'fold1':[glob.glob(f'{save_dir}/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold1_0.8695_epoch319.pt*')[0],
                        glob.glob(f'{save_dir}/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold1_0.8692_epoch384.pt*')[0]],\
               'fold2':[glob.glob(f'{save_dir}/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold2_0.8901_epoch122.pt*')[0],
                        glob.glob(f'{save_dir}/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold2_0.8862_epoch132.pt*')]}
modelDCTList

{'fold0': ['/raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold0_0.8423_epoch197.pth',
  '/raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold0_0.8422_epoch115.pth'],
 'fold1': ['/raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold1_0.8695_epoch319.pth',
  '/raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold1_0.8692_epoch384.pth'],
 'fold2': ['/raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold2_0.8901_epoch122.pth',
  ['/raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchB

In [42]:
start_inference = True

In [ ]:
n_splits = 3
aDCTResultList = list()
if start_inference:
    #### Running 10 folds
    for i in range(0, n_splits):
        

        
        infer_files_fld = copy.deepcopy(BraTS20SubjectsIDHTestDCT[f'fold{i}'])
        
        infer_dataset_fld = monai.data.Dataset(data=infer_files_fld, transform=val_transforms)
       
        infer_loader_fld = monai.data.DataLoader(infer_dataset_fld, batch_size=1, shuffle=False) #num_workers=2, pin_memory=True
        
        aDCTResult = inferWithTA(data_loader = infer_loader_fld, listmodels=modelDCTList[f'fold{i}'], prediction_folder=prediction_folder, topk=len(modelDCTList[f'fold{i}']), num_channels=4,\
                    orientation="LPS", withoptimizer = False, softmaxEnsemble= True, tta = True)
        aDCTResult['TestSplitName'] = f"split{i}"
        aDCTResultList.append(aDCTResult.copy())
        
    
    start_inference = False

available model file: /raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold0_0.8423_epoch197.pth.
available model file: /raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold0_0.8422_epoch115.pth.


Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         239.00209204]
  0%|                                                                                           | 0/71 [00:00<?, ?it/s]

Model 0, /raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold0_0.8423_epoch197.pth is running now
Model 1, /raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold0_0.8422_epoch115.pth is running now


  1%|█▏                                                                                 | 1/71 [00:08<10:27,  8.96s/it]

Model 0, /raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold0_0.8423_epoch197.pth is running now
Model 1, /raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold0_0.8422_epoch115.pth is running now


...

Model 0, /raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold1_0.8695_epoch319.pth is running now
Model 1, /raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold1_0.8692_epoch384.pth is running now


 57%|██████████████████████████████████████████████▋                                   | 41/72 [04:29<03:30,  6.78s/it]

Model 0, /raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold1_0.8695_epoch319.pth is running now
Model 1, /raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold1_0.8692_epoch384.pth is running now


 58%|███████████████████████████████████████████████▊                                  | 42/72 [04:34<03:05,  6.19s/it]

Model 0, /raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold1_0.8695_epoch319.pth is running now
Model 1, /raid/brats2021/pthBraTS2020_IDHGenomics/DynUNetVariants_Brats20/DynUNet_BratsTCGA_3CV_4Chnls1PatchSWIRngr21_2nclass_MorePatchBNormEp500_Fold1_0.8692_epoch384.pth is running now


In [ ]:
print('Result on the testset (3 testsest from 3 splits)')
DFResult = pd.DataFrame.from_dict(aDCTResultList)
display(DFResult)
DFResult.describe()

### Extracting validation set

In [62]:
# current fold: 0 current epoch: 207 dice_score: 0.8495 acc_metric: 0.8371 accuracy: 0.8451, f1score: 0.8451 epoch 207 average training loss: 0.3055 average validation loss: 0.8860 
# current fold: 1 current epoch: 75 dice_score: 0.8634 acc_metric: 0.8944 accuracy: 0.9014, f1score: 0.9014 epoch 75 average training loss: 0.8292 average validation loss: 0.8694
# current fold: 2 current epoch: 482 dice_score: 0.8833 acc_metric: 0.8719 accuracy: 0.8750, f1score: 0.8750 epoch 482 average training loss: 0.1534 average validation loss: 0.8354

print('Result on the validation set (3 validationsets from 3 splits)')
valDF = pd.DataFrame.from_dict({'ValSplitName':['vsplit0', 'vsplit1', 'vsplit2'],  "dice_score": [0.8495, 0.8634, 0.8833],  'acc_metric':[0.8371, 0.8944, 0.8719], \
                                "accuracy": [0.8451, 0.9014, 0.8750],  'f1score':[0.8451, 0.9014, 0.8750], 'NanSubjectNos':[0, 0, 0]})
display(valDF)
valDF.describe()

Result on the validation set (3 validationsets from 3 splits)


,ValSplitName,dice_score,acc_metric,accuracy,f1score,NanSubjectNos
0,vsplit0,0.8495,0.8371,0.8451,0.8451,0
1,vsplit1,0.8634,0.8944,0.9014,0.9014,0
2,vsplit2,0.8833,0.8719,0.8750,0.8750,0


,dice_score,acc_metric,accuracy,f1score,NanSubjectNos
count,3.000000,3.000000,3.000000,3.000000,3.0
mean,0.865400,0.867800,0.873833,0.873833,0.0
std,0.016989,0.028869,0.028168,0.028168,0.0
min,0.849500,0.837100,0.845100,0.845100,0.0
25%,0.856450,0.854500,0.860050,0.860050,0.0
50%,0.863400,0.871900,0.875000,0.875000,0.0
75%,0.873350,0.883150,0.888200,0.888200,0.0
max,0.883300,0.894400,0.901400,0.901400,0.0


In [46]:
# from sklearn.datasets import make_multilabel_classification
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.linear_model import LogisticRegression
# X, y = make_multilabel_classification(random_state=0, n_classes=2)
# inner_clf = LogisticRegression(solver="liblinear", random_state=0)
# clf = MultiOutputClassifier(inner_clf).fit(X, y)
# y_score = np.transpose([y_pred[:, 1] for y_pred in clf.predict_proba(X)])
# roc_auc_score(y, y_score, average=None)

In [ ]:
# roc_auc_score(y, y_score)

In [ ]:
def one_hot_permute(x):
    return F.one_hot(x.squeeze(dim=0).long(), num_classes=3).permute(2, 0, 1)

In [ ]:
xc= torch.tensor([0, 1, 0, 0, 2, 3, 10])
if torch.any(torch.eq(xc, 11)):
    print('Do')

In [ ]:
xc = torch.tensor([0, 100, 500, 10000, 5])
torch.argmax(xc)

In [ ]:
torch.argsort(xc)[-1]

In [ ]:
import torch
is_onehot = True
img = torch.ones((1, 64, 64, 64))
is_onehot2 = img.shape[0] > 1 if is_onehot is None else is_onehot
is_onehot2

In [52]:
dd = np.array([[1.]])
dd.shape

(1, 1)

In [55]:
dx = np.stack([dd, dd], axis = 0)
dx.shape

(2, 1, 1)

In [56]:
np.concatenate([dx, dx], axis = 0).shape

(4, 1, 1)

In [99]:
atns1 = torch.tensor([100, 100, 100, 100])
atns2 = torch.tensor([5, float('nan'), -10])

torch.argmax(atns1)

tensor(0)

In [118]:
auc_metric = ROCAUCMetric()
#y_pred = torch.tensor([float('NaN'), 0, 1, 0, 1, float('NaN')])
y_pred = torch.tensor([float('NaN'), float('NaN'), float('NaN'), float('NaN'), float('NaN'), 1])
y = torch.tensor([0, 0, 0, 0, 0, 1])

if torch.all(torch.isnan(y_pred))==True:
    print('acc_metric#', np.nan, ', auc#', np.nan, ', f1#', np.nan, '\n')
else:
    

    num_nanvalues = torch.isnan(y_pred).sum().item()
    not_nanmask = torch.logical_not(torch.isnan(y_pred))
    y = y[not_nanmask]
    y_pred = y_pred[not_nanmask]
    acc_value = torch.eq(y_pred, y)
    acc_metric = acc_value.sum().item() / len(acc_value)
   
    
    
    auc_metric(y_pred, y)
    auc_result = auc_metric.aggregate()
    auc_metric.reset()

    accscore = balanced_accuracy_score(y, y_pred)
    f1score = f1_score(y, y_pred, average='micro')
    print('acc_metric#', acc_metric, ', auc#', auc_result, ', f1#', f1score, '\n')
    
    
    


acc_metric# 1.0 , auc# nan , f1# 1.0 



y values can not be all 1, skip AUC computation and return `Nan`.


In [101]:


#y_onehot = [post_label(i) for i in torch.unbind(y, dim=0)]
#y_pred_act = [post_pred(i) for i in torch.unbind(y_pred, dim=0)]

